# Imports

In [ ]:
# For quantum simulations
import qutip
from qutip import *
# For FFT
import scipy
import time
import numpy as np
import networkx as nx
import matplotlib as mpl
import matplotlib.pyplot as plt
import itertools
import cmath
import os
from fractions import Fraction
from numpy import linalg as LA

In [ ]:
# Initialize RevTex font
mpl.rcParams['font.family'] = 'STIXGeneral'
mpl.rcParams["font.serif"] = "STIX"
mpl.rcParams["mathtext.fontset"] = "stix"

# Free fermion model definitions

In [ ]:
from scipy.linalg import logm, expm
from sympy import Matrix
from pfapack import pfaffian as pf
import sympy
from IPython.display import display, Latex
sympy.init_printing(use_latex='mathjax')
from numpy.linalg import matrix_power

In [ ]:
# Majorana formalism
def h_mat(L,mu,t):
    # IMPORTANT:
    # t,mu are the H hamiltonian values, they represent the -tZ_1Z_2 and -mu X and so in the Bosonic picture
    # From our calculations in the notes we have to multiply both by 4 to get h hamiltonian: h_parameters = 4H_parameters
    delta = t
    # mu, t, delta as in the notes
    mat = np.zeros((2*L,2*L),dtype=np.complex128)
    omega_plus = t+delta
    omega_minus = t-delta
    for i in range(2*L):
        if i % 2 == 0:
            if i+1 < 2*L:
                mat[i,i+1] = mu/2
            if i+3 < 2*L:
                # there is a factor 1/2 before tunneling terms
                mat[i,i+3] = omega_minus/4
        else:
            if i+1 < 2*L:
                # there is a factor 1/2 before tunneling terms
                mat[i,i+1] = omega_plus/4
    return 1j*(mat-mat.T)

In [ ]:
# Majorana formalism
def h_mat_disorder(L,mu,t):
    # IMPORTANT:
    # t,mu are the H hamiltonian values, they represent the -tX_1X_2 and -mu Z and so in the Bosonic picture
    # From our calculations in the notes we have to multiply both by 4 to get h hamiltonian: h_parameters = 4H_parameters
    delta = t
    # mu, t, delta as in the notes
    mat = np.zeros((2*L,2*L),dtype=np.complex128)
    omega_plus = t+delta
    omega_minus = t+(-1)*delta
    cur_ind_mu = 0
    cur_ind_om_minus = 0
    cur_ind_om_plus = 0
    for i in range(2*L):
        if i % 2 == 0:
            if i+1 < 2*L:
                mat[i,i+1] = mu[cur_ind_mu]/2
                cur_ind_mu += 1
            if i+3 < 2*L:
                # there is a factor 1/2 before tunneling terms
                mat[i,i+3] = omega_minus[cur_ind_om_minus]/4
                cur_ind_om_minus += 1
        else:
            if i+1 < 2*L:
                # there is a factor 1/2 before tunneling terms
                mat[i,i+1] = omega_plus[cur_ind_om_plus]/4
                cur_ind_om_plus += 1
    return 1j*(mat-mat.T)

In [ ]:
# TEST DIRECTION OF DOT
M1 = np.array([1,0,0,0]).reshape((2,2))
M2 = np.array([0,1,0,0]).reshape((2,2))
np.dot(M1,M2)

In [ ]:
def combine_h_exps(_exp_h1,_exp_h2):
    # THIS APPLIES FIRST exp_h1 and the exp_h2 so U = exp_h2 * exp_h1 * \ket{\psi}
    return np.dot(_exp_h2,_exp_h1)
def get_mat_maj_0(syssize,is_edge_minus=False):
    mat_cor_majorana_res = np.zeros((2*syssize,2*syssize), dtype=np.complex128)
    for i in range(2*syssize):
        mat_cor_majorana_res[i,i] = 1
        if i%2==0:
            mat_cor_majorana_res[i,i+1] = 1j
            mat_cor_majorana_res[i+1,i] = -1j
            # if i is one of the edges - need to add minus sign
            if is_edge_minus and (i == 0 or i == 2*syssize-2):
                mat_cor_majorana_res[i,i+1] = -1*mat_cor_majorana_res[i,i+1]
                mat_cor_majorana_res[i+1,i] = -1*mat_cor_majorana_res[i+1,i]
    return mat_cor_majorana_res
def get_zz_mat_maj_0(dist_x):
    '''
    The main difference in this function is that we start from i,j=1 (which is odd)
    '''
    mat_cor_majorana_res = np.zeros((2*dist_x,2*dist_x), dtype=np.complex128)
    for i in range(2*dist_x):
        mat_cor_majorana_res[i,i] = 1
        # If i is odd, the correlation are even as there is +1 shift
        if i%2==1:
            mat_cor_majorana_res[i,i+1] = 1j
            mat_cor_majorana_res[i+1,i] = -1j
    return mat_cor_majorana_res
def exp_cor_op(i1,i2,h_eff, T=None, mat_cor_majorana_0 = None, is_h_exp = False):
    # TESTED - SEEMS TO WORK FINE. RETURNS REASONABLE OUTPUTS.
    '''
    i1 and i2 are indices that start at 0!
    h_eff is the effective single particle Hamiltonian that evolves the state in the Majorana languag
    such that we are calculating <\psi|\gamma_i1 \gamma_i2|\psi>, 
    where |\psi> = U|0> = e^{-i H_eff}|0>, H_eff = \sum \gamma_i (h_eff)_{ij} \gamma_j 
    and |0> is the zero fermionic state
    '''
    # This is the matrix that holds <0|gamma_i\gamma_j|0> 
    if mat_cor_majorana_0 is None:
        mat_cor_majorana_0 = get_mat_maj_0(L)
    if h_eff is None:
        # Returns the vacuum expectation value
        return mat_cor_majorana_res[i1,i2]
    
    N = h_eff.shape[0] # This is the number of Majorana fermions = 2L
    # First we calculate e^{i 4h_eff}, h_eff indeed satisfies h_eff = -h_eff.T
    if T is None:
        if is_h_exp:
            T = matrix_power(h_eff,-1) # Should be e^{-i h_eff}^(-1) = expm(1j*4*A)
        else:
            T = expm(1j*4*h_eff) # expm(1j*4*A)
    
    #B1 = np.zeros((N,1))
    #B2 = np.zeros((N,1))
    # Initializing B defined in the notes. 
    # B is a 1D vector representing the operator O = \sum_i B_i \gamma_i, where \gamma_i is the usual Majorana operator.
    #B1[i1] = 1 
    #B2[i2] = 1
    
    # According to the notes, we can add U^\daggerU between the Majorana operators and then use formulas from the notes
    #B1_res = T[:,i1]#T.dot(B1)
    #B2_res = T[:,i2]#T.dot(B2)
    
    # mat_cor = M is the matrix that holds the coefficients for the vacuum averages 
    # such that M_{ij} <0|gamma_i\gamma_j|0> needs to be added to the result
    # TODO: optimize here the case that the calculation of mat_cor(i1,i2) for i1=i2 or i1=i2+1 or i1=i2-1, as all these cases result in 0
    mat_cor = np.outer(T[:,i1].T,T[:,i2])
    cor_res = np.sum(np.multiply(mat_cor,mat_cor_majorana_0))
    
    # return <\psi | \gamma_{i1} \gamma_{i2} | \psi>
    return cor_res
def get_cor_mat(subsystem_size, h_total, is_h_exp = False, is_edge_minus = False):
    # create correlation matrix for h_total
    M_cor = np.zeros((2*subsystem_size,2*subsystem_size), dtype=np.complex128)
    if h_total is None:
        M_cor = get_mat_maj_0(subsystem_size,False)
        # Returns the free correlation anti symmetrized matrix
        for i in range(2*subsystem_size):
            M_cor[i,i] = 0
        # Minus signs for the edges
        if is_edge_minus:
            M_cor[0,1] *= -1
            M_cor[1,0] *= -1
            if subsystem_size == L and L>1:
                M_cor[2*L-2,2*L-1] *= -1
                M_cor[2*L-1,2*L-2] *= -1
        return M_cor
    if is_h_exp:
        precalc_T = matrix_power(h_total,-1)
    else:
        precalc_T = expm(1j*4*h_total)
    precalc_mat_cor_majorana_0 = get_mat_maj_0(L,is_edge_minus)
    # Iterate over rows
    for i in range(2*subsystem_size):
        # Iterate over columns
        for j in range(2*subsystem_size):
            if i==j or i>j:
                # i=j is the diagonal, which is set to zero
                # i>j are omitted as the matrix is anti-symmetric
                continue
            # this i is from factors of the parity
            M_cor[i,j] = exp_cor_op(i,j,h_total, T= precalc_T, mat_cor_majorana_0 = precalc_mat_cor_majorana_0, is_h_exp=is_h_exp)
            # This matrix is anti-symmetric
            M_cor[j,i] = -1*M_cor[i,j]
    # remove diagonal elements
    for i in range(2*subsystem_size):
        M_cor[i,i] = 0
    return M_cor
def get_zz_mat(start_site, x_dist, h_total, is_h_exp = False):
    # create correlation matrix for h_total
    M_cor = np.zeros((2*x_dist,2*x_dist), dtype=np.complex128)
    if h_total is None:
        M_cor = get_zz_mat_maj_0(x_dist,False)
        # Returns the free correlation anti symmetrized matrix
        for i in range(2*x_dist):
            M_cor[i,i] = 0
        return M_cor
    if is_h_exp:
        precalc_T = matrix_power(h_total,-1)
    else:
        precalc_T = expm(1j*4*h_total)
    precalc_mat_cor_majorana_0 = get_mat_maj_0(L,False)
    # Iterate over rows
    for i in range(2*x_dist):
        # Iterate over columns
        for j in range(2*x_dist):
            if i==j or i>j:
                # i=j is the diagonal, which is set to zero
                # i>j are omitted as the matrix is anti-symmetric
                continue
            # this i is from factors of the parity
            M_cor[i,j] = exp_cor_op(i+2*start_site+1,j+2*start_site+1,h_total, T= precalc_T, mat_cor_majorana_0 = precalc_mat_cor_majorana_0, is_h_exp=is_h_exp)
            # This matrix is anti-symmetric
            M_cor[j,i] = -1*M_cor[i,j]
    # remove diagonal elements
    for i in range(2*x_dist):
        M_cor[i,i] = 0
    return M_cor
def get_zz_cor_arr(start_site, max_dist, h_total, is_h_exp = False):
    M = get_zz_mat(start_site,max_dist,h_total,is_h_exp)
    zz_arr = []
    for r in range(1,max_dist+1):
        A = M[:2*(r),:2*(r)]
        cur_cor = ((-1j)**(r))*pf.pfaffian(0.5*(A-A.T))
        zz_arr.append(cur_cor)
    return zz_arr
def get_cor_single_exp(system_size,h_total_exp):
    precalc_mat_cor_majorana_0 = get_mat_maj_0(system_size,False)
    if h_total_exp is None:
        # No Hamiltonian, we calculate the expected values on the simple two pairs
        arr_single_pars = np.array([-1j*precalc_mat_cor_majorana_0[2*i,2*i+1] for i in range(system_size)])
    else:
        precalc_T = matrix_power(h_total_exp,-1)
        # The -1j fixes the prefactors and results with the real <X_i>
        arr_single_pars = np.array([-1j*exp_cor_op(2*i,2*i+1,h_total_exp, T= precalc_T, mat_cor_majorana_0 = precalc_mat_cor_majorana_0, is_h_exp=True) for i in range(system_size)])
    # Return the expected parities over all the single qubits. Each quit is characterized by 2 Majorana fermions, 
    # where each of the Majorana fermions has an even index at the start. 
    # Thus, the pairs are consucutive. Returns the array arr, where arr[i] = <X_i>
    return np.real(arr_single_pars)
def get_cor_dw_exp(system_size,h_total_exp):
    precalc_mat_cor_majorana_0 = get_mat_maj_0(system_size,False)
    if h_total_exp is None:
        # No Hamiltonian, we calculate the expected values on the simple two pairs
        arr_dws = np.array([-1j*precalc_mat_cor_majorana_0[2*i+1,2*(i+1)] for i in range(system_size-1)])
    else:
        precalc_T = matrix_power(h_total_exp,-1)
        # The -1j fixes the prefactors and results with the real <X_i>
        arr_dws = np.array([-1j*exp_cor_op(2*i+1,2*(i+1),h_total_exp, T= precalc_T, mat_cor_majorana_0 = precalc_mat_cor_majorana_0, is_h_exp=True) for i in range(system_size-1)])
    # Return the expected <Z_iZ_{i+1}> over all the pairs.
    # Thus, the pairs are consucutive. Returns the array arr, where arr[i] = <Z_iZ_{i+1}>
    return np.real(arr_dws)

# Adabatic sweeping functions and data saving

In [ ]:
# Steps are the same definitions as in the draft
# step_num=10
# l = np.linspace(0,np.pi/2,step_num+2)[1:-1]
# print(l)
# l = [np.pi*(i/(2*(step_num+1))) for i in range(1,step_num+1)]
# print(l)

In [ ]:
def calc_dws_using_exp_triv_to_ferro_end_only(cur_size,N_steps,debug_print=True,every_disorder_var_z=0,every_disorder_var_xx=0,static_disorder_var_z =0,static_disorder_var_xx = 0, calc_cors=False,rad=1,trot=False):
    # This should be more efficient and exact compared with calc_ent_res
    # TODO: test if this generates the same results as calc_ent_res
    # TODO: FIX THIS GLOBAL USE OF L IN THE FUNCTIONS AND REMOVE IT
    global L
    L=cur_size
    half_sys = int(L/2)
    # UPDATE 27.03: Throw exception or Trotterization
    if trot:
        raise Exception("Check exact definitions for Trotterization and N_steps")
        steps_list = np.array(list(range(1,N_steps+1)))
        dt =0.1
        curT = 0.5*dt*N_steps
    else:
        # UPDATE 27.03: We remove the first and last steps
        # N_steps -> N_steps+2 as we want the old partitions, but we remove the first and last step
        steps_list = np.linspace(0,np.pi/2,N_steps+2)[1:-1]

    # Though the array doesn't contain complex numbers, we refer to them as complex for simple multiplication with complex numbers
    total_dw_end = 0
    # TODO: Parametrize this variable
    max_dist = int(L/5)
    zz_cors_end = np.zeros(shape=(max_dist,),dtype=np.float64)
    
    # Disorder parameters
    is_disorder = False
    if every_disorder_var_z is not None or every_disorder_var_xx is not None:
        is_disorder = True
        disorder_every_step = True
    # Static disorder
    if static_disorder_var_z is not None or static_disorder_var_xx is not None:
        is_disorder = True
        static_disorder = True
        stat_disorder_arr_mu =np.random.normal(0, static_disorder_var_z, cur_size)
        stat_disorder_arr_t = np.random.normal(0, static_disorder_var_xx, cur_size-1)
    # This is for the plus state
    cur_step_num = 0
    cur_h_exp = None
    r0 = rad
    debug_step_skip = int(N_steps/10)

    # Going over the path adiabatically
    for step in steps_list:
        if debug_print and cur_step_num % debug_step_skip == 0:
            print(cur_step_num)
        # Calculate the next step parameters
        if trot: # The linear trotterized path
            curg = -1*(2-(step*dt)/curT)
            curJ = -1*(step*dt)/curT
            #s = float(cur_step_num)/float(N_steps)
            alpha_cur = curg*dt # H_{PM} coefficient
            beta_cur =  curJ*dt # H_{FM} coefficient
        else:
            theta = step
            # The half here seems to be artifact of some notation
            alpha_cur = 2*0.5*r0 * np.cos(theta) # H_{PM} coefficient
            beta_cur = 2*0.5*r0*np.sin(theta) # H_{FM} coefficient
            # UPDATE 13.09 changed r_0 to be 1
        if is_disorder:
            disorder_arr_mu = np.zeros(cur_size)
            disorder_arr_t = np.zeros(cur_size-1)
            if static_disorder:
                disorder_arr_mu = disorder_arr_mu + stat_disorder_arr_mu
                disorder_arr_t = disorder_arr_t + stat_disorder_arr_t
            if disorder_every_step:
                disorder_arr_mu = disorder_arr_mu + np.random.normal(0, every_disorder_var_z, cur_size)
                disorder_arr_t = disorder_arr_t + np.random.normal(0, every_disorder_var_xx, cur_size-1) # There are only cur_size-1 pairs
            cur_h_X = h_mat_disorder(cur_size,disorder_arr_mu+alpha_cur,np.zeros(cur_size-1))
            cur_h_ZZ = h_mat_disorder(cur_size,np.zeros(cur_size),disorder_arr_t+beta_cur)              
        else:
            cur_h_X = h_mat(cur_size,alpha_cur,0)
            cur_h_ZZ = h_mat(cur_size,0,beta_cur)
        h_ZZ_exp = expm(-1*1j*4*cur_h_ZZ)
        h_X_exp = expm(-1*1j*4*cur_h_X)
        # start of the loop intializes cur_h
        if cur_h_exp is None:
            # EDIT: CORRECT THIS, START FROM H_FM at EACH STEP as we start from the |+> state
            # First we apply the ZZ interaction H_{FM} with coefficient beta and only then we apply X of H_{PM} wth parameter alpha
            cur_h_exp = combine_h_exps(h_ZZ_exp,h_X_exp) # this is F(\alpha,\beta)
        else:
            # First we apply the ZZ interaction H_{FM} with coefficient beta and only then we apply X of H_{PM} wth parameter alpha
            cur_h_exp = combine_h_exps(cur_h_exp,combine_h_exps(h_ZZ_exp,h_X_exp)) # this is F(\alpha,\beta)
        cur_step_num += 1
    # Calculate the number of domain walls at the end
    arr_dws = get_cor_dw_exp(cur_size,cur_h_exp)
    # TODO: Choose L or L-1, currently we choose L-1
    total_dw_end = 0.5*(cur_size-1-sum(arr_dws))
    #total_dw_end = 0.5*(cur_size-sum(arr_dws))
    # Calculate the correlation array at the end
    if calc_cors:  
        zz_cors_end = np.real(get_zz_cor_arr(half_sys,max_dist,cur_h_exp,is_h_exp=True))
    else:
        zz_cors_end = None
    return total_dw_end,zz_cors_end

In [ ]:
# Repeat the calculation above with more optimized version that can run: Do the calculation in optimized version of 10000 steps adiabaticity for different system sizes.
# ONLY XX DISORDER
import itertools
def save_disorder_data(cur_L_size_arr,cur_N_opt,cur_disorder_xx_var_arr=None,cur_static_disorder_xx_var_arr=None,cur_disorder_count=2,_model_name='highdisorder',radiu=1,to_trotter=False,setNum=-1):
    if cur_static_disorder_xx_var_arr is None:
        cur_static_disorder_xx_var_arr = [0]
    if cur_disorder_xx_var_arr is None:
        cur_disorder_xx_var_arr = [0]
    cur_N_max = cur_N_opt[-1]
    for L_size in cur_L_size_arr:
        dw_sat_end_dict = dict()
        dw_sat_end_std_dict = dict()
        name = 'pt_localization_excitation_L={0}_maxN={1}_system_majorana_mapping_disorder_everystep=True_rad={2}_trotter={3}_{4}_endstep_new_nsteps_definition_corrected_noise'.format(str(L_size),str(cur_N_max),str(radiu),str(to_trotter),_model_name)
        for cur_disorder_var_x in cur_disorder_xx_var_arr:
            if cur_disorder_var_x is None:
                cur_disorder_var_x = 0
            for cur_stat_disorder_var_x in cur_static_disorder_xx_var_arr:
                dw_arr_end = dict()
                dw_arr_end_std = dict()
                dw_arr_full_data = dict()
                for cur_n_opt in cur_N_opt:
                    cur_disorder_arr_res = []
                    #cur_zz_cors_arr = []
                    for cur_dis_ind in range(cur_disorder_count):
                        print(cur_dis_ind)
                        total_dws_val,_ = calc_dws_using_exp_triv_to_ferro_end_only(L_size,int(cur_n_opt),debug_print=False,static_disorder_var_xx=cur_stat_disorder_var_x,static_disorder_var_z=cur_stat_disorder_var_x,every_disorder_var_z=cur_disorder_var_x,every_disorder_var_xx=cur_disorder_var_x,rad=radiu,trot=to_trotter)
                        # TODO: choose L or L-1, currently we choose L-1
                        cur_disorder_arr_res.append(total_dws_val/(L_size-1))
                        # Currently try L
                    print(cur_n_opt)
                    #cor_len_per_point_arrs =  obtain_cor_arr(cur_zz_cors_arr)
                    dw_arr_full_data[cur_n_opt] = np.array(cur_disorder_arr_res)
                    dw_arr_end[cur_n_opt] = np.mean(np.array(cur_disorder_arr_res))
                    if setNum == -1:
                        dw_arr_end_std[cur_n_opt] = np.std(np.array(cur_disorder_arr_res))
                    else:
                        splited_arr = np.array_split(np.array(cur_disorder_arr_res),setNum)
                        avg_arr = np.array([np.mean(splited_arr[i]) for i in range(setNum)])
                        dw_arr_end_std[cur_n_opt] = np.std(avg_arr)
                print(dw_arr_end)
                print(dw_arr_end_std)
                dw_sat_end_dict[(cur_disorder_var_x,cur_stat_disorder_var_x)] = dw_arr_end
                dw_sat_end_std_dict[(cur_disorder_var_x,cur_stat_disorder_var_x)] = dw_arr_end_std
        # DEBUG: Save full data with all counts
        # np.save(name+'_full_data_arr.npy',dw_arr_full_data)
        np.save(name+'_data_mean_dict.npy',dw_sat_end_dict)
        np.save(name+'_data_std_dict.npy',dw_sat_end_std_dict)

# Plot function definitions

In [ ]:
FONTSIZE_LEGEND = 22
FONTSIZE_AXIS = 24
FONTSIZE_TICKS = 20
FONTSIZE_TITLE = 16

def plot_kz_scaling_disorder(xaxis,sys_size,max_N,model_name,rad=1,trot=False):
    # Update 27.03.22: Remove trot variable usage and add to the filenames the new definiton
    # Also, only output pdf files
    # Load data from files
    if trot:
        raise Exception("Check exact definitions for Trotterization and N_steps")
    try:
        load_filename = 'pt_localization_excitation_L={0}_maxN={1}_system_majorana_mapping_disorder_everystep=True_rad={2}_trotter={3}_{4}'.format(str(sys_size),str(max_N),str(rad),str(trot),model_name)
        yaxis_dict = np.load(load_filename+'_endstep_new_nsteps_definition_data_mean_dict.npy', allow_pickle=True).item()
        yaxis_std_dict = np.load(load_filename+'_endstep_new_nsteps_definition_data_std_dict.npy', allow_pickle=True).item()
    except (FileNotFoundError):
        load_filename = 'pt_localization_excitation_L={0}_maxN={1}_system_majorana_mapping_disorder_everystep=True_{2}'.format(str(sys_size),str(max_N),model_name)
        yaxis_dict = np.load(load_filename+'_endstep_new_nsteps_definition_data_mean_dict.npy', allow_pickle=True).item()
        yaxis_std_dict = np.load(load_filename+'_endstep_new_nsteps_definition_data_std_dict.npy', allow_pickle=True).item()

    # Choose the xaxis to be the N's of the first in the dictionary
    if xaxis is None:
        xaxis = list(list(yaxis_dict.values())[0].keys())
        print(xaxis)

    fig, ax = plt.subplots(figsize=(10, 6))
    ax.set_xticks(xaxis) # NO MINOR BOOL FOR NEWER MATPLOTLIB
    # TODO: We divide N_{ex} by L-1, change to L in the case of periodic boundary conditions
    ax.set_ylabel(r'Excitations density $\frac{N_{\rm ex}}{L}$',fontsize=FONTSIZE_AXIS)
    ax.set_xlabel(r'$N_{\rm steps}$',fontsize=FONTSIZE_AXIS)
    
    # Set log scale
    ax.set_yscale('log')
    ax.set_xscale('log')  

    plt.tight_layout()

    for k,yaxis in sorted(yaxis_dict.items()):
        yerr_arr = yaxis_std_dict[k]
        l_yaxis = []
        l_yerr_arr = []
        for n_opt in xaxis:
            l_yaxis.append(yaxis[n_opt])
            l_yerr_arr.append(yerr_arr[n_opt])
        print(l_yaxis)
        ax.errorbar(xaxis,l_yaxis,ls='--',label=r'$\sigma='+str(k)+r'$',marker='o',markersize=4,capsize=2,yerr=l_yerr_arr)
        fit_params = scipy.optimize.curve_fit(lambda x,a,b: a+b*x, np.log(np.array(xaxis[:])[:]), np.log(np.array(l_yaxis[:])[:]))
        fit_slope = fit_params[0][1]
        #print('yaxis:'+str(np.log(np.array(yaxis[:])[:])))
        print('sigma={0} with slope={1}'.format(str(k), str(fit_slope)))
    # TODO: Unique names
    name = 'kz_scaling_'+str(sys_size)+'system_fermion_mapping_disorder_everystep=True_optimized_new_version'

    ax.legend(loc='lower left',fontsize=FONTSIZE_LEGEND)
    #ax.figure.savefig(name+'.png')
    #ax.figure.savefig(name+'.svg')
    ax.figure.savefig(name+'.pdf')

In [ ]:
FONTSIZE_LEGEND = 12
FONTSIZE_AXIS = 24
FONTSIZE_TICKS = 20
FONTSIZE_TITLE = 16

def plot_kz_scaling_disorder_static_new_format(xaxis,sys_size,max_N,model_name,rad=1,trot=False):
    # Update 27.03.22: Remove trot variable usage and add to the filenames the new definiton
    # Also, only output pdf files
    # Load data from files
    if trot:
        raise Exception("Check exact definitions for Trotterization and N_steps")
    try:
        load_filename = 'pt_localization_excitation_L={0}_maxN={1}_system_majorana_mapping_disorder_everystep=True_rad={2}_trotter={3}_{4}'.format(str(sys_size),str(max_N),str(rad),str(trot),model_name)
        yaxis_dict = np.load(load_filename+'_endstep_new_nsteps_definition_data_mean_dict.npy', allow_pickle=True).item()
        yaxis_std_dict = np.load(load_filename+'_endstep_new_nsteps_definition_data_std_dict.npy', allow_pickle=True).item()
    except (FileNotFoundError):
        load_filename = 'pt_localization_excitation_L={0}_maxN={1}_system_majorana_mapping_disorder_everystep=True_{2}'.format(str(sys_size),str(max_N),model_name)
        yaxis_dict = np.load(load_filename+'_endstep_new_nsteps_definition_data_mean_dict.npy', allow_pickle=True).item()
        yaxis_std_dict = np.load(load_filename+'_endstep_new_nsteps_definition_data_std_dict.npy', allow_pickle=True).item()

    # Choose the xaxis to be the N's of the first in the dictionary
    if xaxis is None:
        xaxis = list(list(yaxis_dict.values())[0].keys())
        print(xaxis)

    fig, ax = plt.subplots(figsize=(10, 6))
    ax.set_xticks(xaxis) # NO MINOR BOOL FOR NEWER MATPLOTLIB
    # TODO: We divide N_{ex} by L-1, change to L in the case of periodic boundary conditions
    ax.set_ylabel(r'Excitations density $\frac{N_{\rm ex}}{L}$',fontsize=FONTSIZE_AXIS)
    ax.set_xlabel(r'$N_{\rm steps}$',fontsize=FONTSIZE_AXIS)
    
    # Set log scale
    ax.set_yscale('log')
    ax.set_xscale('log')  

    plt.tight_layout()

    for k,yaxis in sorted(yaxis_dict.items()):
        every_disorder = k[0]
        static_disorder = k[1]
        yerr_arr = yaxis_std_dict[k]
        l_yaxis = []
        l_yerr_arr = []
        for n_opt in xaxis:
            l_yaxis.append(yaxis[n_opt])
            l_yerr_arr.append(yerr_arr[n_opt])
        print(l_yaxis)
        ax.errorbar(xaxis,l_yaxis,ls='--',label=r'$\sigma_{\rm step}='+str(every_disorder)+r', \sigma_{\rm static}='+str(static_disorder)+r' $',marker='o',markersize=4,capsize=2,yerr=l_yerr_arr)
        fit_params = scipy.optimize.curve_fit(lambda x,a,b: a+b*x, np.log(np.array(xaxis[:])[:]), np.log(np.array(l_yaxis[:])[:]))
        fit_slope = fit_params[0][1]
        #print('yaxis:'+str(np.log(np.array(yaxis[:])[:])))
        print('sigma={0} with slope={1}'.format(str(k), str(fit_slope)))
    # TODO: Unique names
    name = 'plot_'+load_filename#'kz_scaling_'+str(sys_size)+'system_fermion_mapping_disorder_everystep=True_optimized_new_version_new_format_with_static_disorder_labels'

    ax.legend(loc='lower left',fontsize=FONTSIZE_LEGEND)
    ax.figure.savefig(name+'.pdf')
    #ax.figure.savefig(name+'.svg')

# Adding Disorder

## Static disorder with constant every step disorder

In [ ]:
# Define variables
disorder_xx_var_arr = [3e-2]
static_disorder_xx_var_arr = [1e-3,1e-2,3e-2,5e-2,1e-1,3e-1,5e-1]
max_N = 100#30
N_start = 1
radi = 1
to_trotter_radius = False
N_opt = list(range(N_start,max_N+1,1))
L_sizes = [50]
disorder_count = 10
is_disorder_every_step = False
model_name = 'static_disorder_with_everystep=0.03'
print(N_opt)

In [ ]:
save_disorder_data(L_sizes,N_opt,disorder_xx_var_arr,static_disorder_xx_var_arr,cur_disorder_count=disorder_count,_model_name=model_name,radiu=radi,to_trotter= to_trotter_radius)

In [ ]:
# Plot the Excitations
for cur_sys_size in L_sizes:
    # TODO: save them with actual names
    plot_kz_scaling_disorder_static_new_format(N_opt,cur_sys_size,N_opt[-1],model_name,rad=radi,trot=to_trotter_radius)

## Every step disorder with constant static disorder

In [ ]:
# Define variables
disorder_xx_var_arr = [1e-3,1e-2,3e-2,5e-2,1e-1,3e-1,5e-1]
static_disorder_xx_var_arr = [3e-1]
max_N = 100#30
N_start = 1
radi = 1
to_trotter_radius = False
N_opt = list(range(N_start,max_N+1,1))
L_sizes = [50]
disorder_count = 30
model_name = 'everystep_disorder_with_static='+str(static_disorder_xx_var_arr[0])
print(N_opt)
print(model_name)

In [ ]:
save_disorder_data(L_sizes,N_opt,disorder_xx_var_arr,static_disorder_xx_var_arr,cur_disorder_count=disorder_count,_model_name=model_name,radiu=radi,to_trotter= to_trotter_radius)

In [ ]:
# Plot the Excitations
for cur_sys_size in L_sizes:
    # TODO: save them with actual names
    plot_kz_scaling_disorder_static_new_format(N_opt,cur_sys_size,N_opt[-1],model_name,rad=radi,trot=to_trotter_radius)

## General structure for obtaining data

In [ ]:
# Define variables
disorder_xx_var_arr = [1e-2]#[1e-2,1e-2,3e-2,5e-2,1e-1,3e-1,5e-1,1]
static_disorder_xx_var_arr = [0,5e-3,1e-2,5e-2,1e-1]
max_N = 600#30
N_start = 20
radi = 1 # IGNORE THIS CURRENTLY
to_trotter_radius = False
N_opt = list(range(N_start,max_N+1,25))
L_sizes = [40]
disorder_count = 6
is_disorder_every_step = True
model_name = 'static_and_every_step_disorder'
print(N_opt)

In [ ]:
# No disorder
save_disorder_data(L_sizes,N_opt,disorder_xx_var_arr,static_disorder_xx_var_arr,cur_disorder_count=disorder_count,_model_name=model_name,radiu=radi,to_trotter= to_trotter_radius)

In [ ]:
# Plot the Excitations
for cur_sys_size in L_sizes:
    plot_kz_scaling_disorder_static_new_format(N_opt,cur_sys_size,N_opt[-1],model_name,rad=radi,trot=to_trotter_radius)

In [ ]:
d = {100: 0.06758624097594657, 200: 0.04675348106663357, 300: 0.03752408889092279, 400: 0.03201145820373713, 500: 0.02823126496497874, 600: 0.0254294377277694, 700: 0.023255201721835594, 800: 0.021503561198011445, 900: 0.020055099076554228, 1000: 0.018828253510122017, 1100: 0.017770106962242294, 1200: 0.016847583010850375, 1300: 0.016033498956973702, 1400: 0.015309698768704472, 1500: 0.014659320928273045, 1600: 0.014069913264894596, 1700: 0.013533342072178656, 1800: 0.013041787013080619, 1900: 0.012590369835634385, 2000: 0.01217325156981508, 2100: 0.011785970754115582, 2200: 0.011425326924183521, 2300: 0.011088104386267617, 2400: 0.010772678202906394, 2500: 0.010476381873587293}
for k in d:
    d[k] = d[k] + (1/(2*128))
print(d)

## Verfication data for $L=4,5$ without disorder

In [ ]:
# Define variables
disorder_xx_var_arr = [0]#[1e-2,1e-2,3e-2,5e-2,1e-1,3e-1,5e-1,1]
max_N = 30#300
N_start = 1
N_opt = list(range(N_start,max_N+1,1))
L_sizes = [4]
disorder_count = 1
is_disorder_every_step = True
to_trotter_radius = False
N_opt

In [ ]:
# Save the data
model_name = 'no_disorder_verfication_test_to_delete'
save_disorder_data(L_sizes,N_opt,disorder_xx_var_arr,cur_disorder_count=disorder_count,_model_name=model_name,radiu=radi,to_trotter= to_trotter_radius)

In [ ]:
# Plot the Excitations
for cur_sys_size in L_sizes:
    plot_kz_scaling_disorder([int(i) for i in N_opt[5:]],cur_sys_size,N_opt[-1],model_name,rad=1,trot=to_trotter_radius)

In [ ]:
(abs(np.array(means[100:])-np.array(free_fermion_dim_res[100:])))/(np.array(means[100:]))


In [ ]:
means

In [ ]:
free_fermion_dim_res

In [ ]:
np.array(devs)

In [ ]:
d={2: 0.3078018659785016, 3: 0.28028028060625443, 4: 0.2508101277413441, 5: 0.22123924788454963, 6: 0.19331861835893638, 7: 0.1682735798460719, 8: 0.1466935826662156, 9: 0.12859618729101543, 10: 0.11359565087588182, 11: 0.10110974720731943, 12: 0.09054531563144312, 13: 0.081421713982558, 14: 0.07341727262636095, 15: 0.0663490536208479, 16: 0.060113521915755884, 17: 0.054621200385468395, 18: 0.04975228387582017, 19: 0.04534644001622312, 20: 0.0412244453829231, 21: 0.03722745348943753, 22: 0.03325503597973493, 23: 0.029286161840575458, 24: 0.02537571246591212, 25: 0.021629119187694463, 26: 0.018165394293336263, 27: 0.01508172380136715, 28: 0.0124305102150708, 29: 0.010213863952553007, 30: 0.008393732216180227, 31: 0.006910785165166559, 32: 0.005703488113602255, 33: 0.004720598890155118, 34: 0.003924400215581307, 35: 0.0032863574508054474, 36: 0.0027797767710430557, 37: 0.0023744657147835024, 38: 0.0020365341001233883, 39: 0.0017334168238287335, 40: 0.0014414278393460138, 41: 0.0011518705193151273, 42: 0.0008723874499081777, 43: 0.0006224115742550218, 44: 0.0004242332884846582, 45: 0.00029313992891329593, 46: 0.0002305022934334655, 47: 0.00022245490839482388, 48: 0.00024452021340008273, 49: 0.00027015763522656816, 50: 0.0002797535882321789, 51: 0.00026656283743046555, 52: 0.00023749398135889757, 53: 0.00020876950984168494, 54: 0.00019848194471131878, 55: 0.00021912470051040542, 56: 0.00027294041634425464, 57: 0.00035158827662902975, 58: 0.00043980701138875605, 59: 0.0005212146476432089, 60: 0.0005837372137296551, 61: 0.000622586164376826, 62: 0.0006399410645906434, 63: 0.0006419588477756625, 64: 0.0006347921042892946, 65: 0.0006215413387499572, 66: 0.0006014508078297324, 67: 0.0005715127631717998, 68: 0.0005295041648532747, 69: 0.0004768426788450686, 70: 0.00041976950646623507, 71: 0.00036815924452075305, 72: 0.0003323488776436623, 73: 0.00031928275004222684, 74: 0.0003296005018182206, 75: 0.0003569227535020018, 76: 0.00038969022340962223, 77: 0.00041487797254935455, 78: 0.0004221720839111172, 79: 0.00040705409965328493, 80: 0.00037172372788655633, 81: 0.0003236766112632387, 82: 0.00027266092145324566, 83: 0.0002272953848820869, 84: 0.00019263757328791264, 85: 0.00016948089620509124, 86: 0.00015537952787640177, 87: 0.0001466960712882178, 88: 0.0001406263600859011, 89: 0.0001362953578063116, 90: 0.00013454116547813774, 91: 0.00013665522359262994, 92: 0.0001428299858489135, 93: 0.00015117427304785677, 94: 0.00015785955747005254, 95: 0.00015840664679474722, 96: 0.00014956895894317293, 97: 0.00013096499658682426, 98: 0.0001056912943679933, 99: 7.956973556858908e-05, 100: 5.9262033591892895e-05, 101: 4.9967460022874256e-05, 102: 5.3601646356519574e-05, 103: 6.816157510799077e-05, 104: 8.850226186241095e-05, 105: 0.00010819027869316915, 106: 0.00012169200528303985, 107: 0.00012606220017641334, 108: 0.00012154554787306242, 109: 0.00011097225960191952, 110: 9.831379018347224e-05, 111: 8.706478346535522e-05, 112: 7.911921348724427e-05, 113: 7.453112068163865e-05, 114: 7.212981036526973e-05, 115: 7.058601553019035e-05, 116: 6.935888451176098e-05, 117: 6.905520566384087e-05, 118: 7.104302479817488e-05, 119: 7.653295380305458e-05, 120: 8.560195483275912e-05, 121: 9.66702730184436e-05, 122: 0.00010673998378470569, 123: 0.00011235696992909865, 124: 0.00011092319130053448, 125: 0.00010181174149611245, 126: 8.68001584158525e-05, 127: 6.961035595139571e-05, 128: 5.4708568479037524e-05, 129: 4.582011943595384e-05, 130: 4.472553078708567e-05, 131: 5.078318158879824e-05, 132: 6.132529693251658e-05, 133: 7.272718556235643e-05, 134: 8.169914133537848e-05, 135: 8.629540522390548e-05, 136: 8.628818741692375e-05, 137: 8.28418237673878e-05, 138: 7.771471077266945e-05, 139: 7.239066396480531e-05, 140: 6.752957022282402e-05, 141: 6.294582337051935e-05, 142: 5.8061148016319564e-05, 143: 5.255643489462022e-05, 144: 4.6863061717676104e-05, 145: 4.22211072997151e-05, 146: 4.024581506778535e-05, 147: 4.21848043854108e-05, 148: 4.820672550650181e-05, 149: 5.706571413588435e-05, 150: 6.633299057183055e-05, 151: 7.314447847521037e-05, 152: 7.518755678478002e-05, 153: 7.153930624485365e-05, 154: 6.302072722730312e-05, 155: 5.1923826633733704e-05, 156: 4.121901246216068e-05, 157: 3.35533299761955e-05, 158: 3.0422296413775325e-05, 159: 3.181314531593612e-05, 160: 3.6415693752900324e-05, 161: 4.226698855717507e-05, 162: 4.7534262607099954e-05, 163: 5.111160383508562e-05, 164: 5.281419108145909e-05, 165: 5.3146104317502996e-05, 166: 5.280613889594813e-05, 167: 5.2198815504223006e-05, 168: 5.119385488749906e-05, 169: 4.924315637892344e-05, 170: 4.578451469276305e-05, 171: 4.0719074029841686e-05, 172: 3.4709169872434074e-05, 173: 2.912215980854782e-05, 174: 2.5607756551434175e-05, 175: 2.5468078877969685e-05, 176: 2.908394451184293e-05, 177: 3.565070222710798e-05, 178: 4.335545800689585e-05, 179: 4.9946768951203424e-05, 180: 5.3485184325722614e-05, 181: 5.298761032449898e-05, 182: 4.872164227250527e-05, 183: 4.204824023634757e-05, 184: 3.489134861082525e-05, 185: 2.905552830967384e-05, 186: 2.5659549223585227e-05, 187: 2.4889412897947476e-05, 188: 2.613023435856417e-05, 189: 2.8377105365295918e-05, 190: 3.071869024588262e-05, 191: 3.26765341193506e-05, 192: 3.426748604717922e-05, 193: 3.579443163156023e-05, 194: 3.74980617620461e-05, 195: 3.926281349890074e-05, 196: 4.0538164362224904e-05, 197: 4.0528881994905674e-05, 198: 3.857501430652258e-05, 199: 3.454539457548034e-05, 200: 2.9051657825740484e-05, 201: 2.3361502081487018e-05, 202: 1.901897967963399e-05, 203: 1.7309333925112174e-05, 204: 1.8779279249314662e-05, 205: 2.3008181104492802e-05, 206: 2.8727365176006714e-05, 207: 3.424540483150847e-05, 208: 3.8015619459893145e-05, 209: 3.912885953893941e-05, 210: 3.755085838733999e-05, 211: 3.403173068032098e-05, 212: 2.9748343795839755e-05, 213: 2.5841930116750806e-05, 214: 2.3042297916312886e-05, 215: 2.151754940589908e-05, 216: 2.098067261313563e-05, 217: 2.0970738707326575e-05, 218: 2.1156418598529864e-05, 219: 2.1511894677657868e-05, 220: 2.2285850959979925e-05, 221: 2.378878684953604e-05, 222: 2.6113878666578216e-05, 223: 2.8941051302429816e-05, 224: 3.15375073253558e-05, 225: 3.2977456120815894e-05, 226: 3.250020785268889e-05, 227: 2.985598666567964e-05, 228: 2.548464733161578e-05, 229: 2.0436851775847604e-05, 230: 1.6052941727240366e-05, 231: 1.3516388331173227e-05, 232: 1.3452679774298796e-05, 233: 1.5727791935852647e-05, 234: 1.952029750285078e-05, 235: 2.3631898838287018e-05, 236: 2.6908760366151085e-05, 237: 2.8608475617734424e-05, 238: 2.857873922645915e-05, 239: 2.719822815470489e-05, 240: 2.5130188534191095e-05, 241: 2.3011819994348908e-05, 242: 2.121811531770786e-05, 243: 1.9793437870285757e-05, 244: 1.8560533975087168e-05, 245: 1.733355277762823e-05, 246: 1.611618828497215e-05, 247: 1.5177641874937938e-05, 248: 1.4961102414057237e-05, 249: 1.5862666011649356e-05, 250: 1.7984667147900406e-05, 251: 2.0985872422755552e-05, 252: 2.4112978426305e-05, 253: 2.6419749870202718e-05, 254: 2.709627614749799e-05, 255: 2.577829957032633e-05, 256: 2.2708897689997325e-05, 257: 1.8681684962107425e-05, 258: 1.4782223508628492e-05, 259: 1.202545096031423e-05, 260: 1.1027533833714761e-05, 261: 1.183409130941134e-05, 262: 1.396108108719929e-05, 263: 1.661832821768054e-05, 264: 1.901530242683543e-05, 265: 2.0623135719969692e-05, 266: 2.1294922734726995e-05, 267: 2.1213749258860066e-05, 268: 2.0713940900307648e-05, 269: 2.0072376766563e-05, 270: 1.9371984525988022e-05, 271: 1.849831310100451e-05, 272: 1.726257180948802e-05, 273: 1.5582145288036326e-05, 274: 1.3620805648928425e-05, 275: 1.1808543832462584e-05, 276: 1.0716918079856086e-05, 277: 1.0834610609133932e-05, 278: 1.2338251099253128e-05, 279: 1.496239339741449e-05, 280: 1.8035069620614408e-05, 281: 2.067728841422441e-05, 282: 2.2095427625619852e-05, 283: 2.1854677301128927e-05, 284: 2.0027198124143258e-05, 285: 1.7158251272464398e-05, 286: 1.406619155641664e-05, 287: 1.1557266210271422e-05, 288: 1.0166648019412547e-05, 289: 1.0021219759940673e-05, 290: 1.0865403524462636e-05, 291: 1.222291217854258e-05, 292: 1.3614110970043095e-05, 293: 1.4732644075884002e-05, 294: 1.5511271764134804e-05, 295: 1.606210567284272e-05, 296: 1.6535037909326533e-05, 297: 1.6973872698685295e-05, 298: 1.7247131005852328e-05, 299: 1.709182556930422e-05, 300: 1.6252026445099748e-05}
free_fermion_dim_res= []
for k in range(2,301):
    free_fermion_dim_res.append(d[k])

In [ ]:
d[163]

## NISQ devices plots

In [ ]:
# Define variables
disorder_xx_var_arr = [1e-3,5e-3,1e-2,5e-2,1e-1,3e-1]#[1e-2,1e-2,3e-2,5e-2,1e-1,3e-1,5e-1,1]
max_N = 1000#30
N_start = 2
count_points = 80
N_opt = list(range(2,30,2))+list(np.linspace(N_start,max_N,count_points).astype(int))
L_sizes = [50]
disorder_count = 20
is_disorder_every_step = True

In [ ]:
# Save the data
model_name = 'nisq_parameters_low_nsteps'
save_disorder_data(L_sizes,N_opt,disorder_xx_var_arr,disorder_count,model_name)

In [ ]:
# Plot the Excitations
model_name = 'merged_nisq_parameters'
for cur_sys_size in L_sizes:
    plot_kz_scaling_disorder(None,cur_sys_size,N_opt[-1],model_name)

## Check scalings of static disorder

In [ ]:
# Copy scaling code from the other file to here
# Plot the saturation values as a function of the disorder
def plot_sat_values(sys_size,max_N,rad=1,trot=False):
    # Load data from files
    try:
        load_filename = 'pt_localization_excitation_L={0}_maxN={1}_system_majorana_mapping_disorder_everystep=True_rad={2}_trotter={3}_{4}'.format(str(sys_size),str(max_N),str(rad),str(trot),model_name)
        yaxis_dict = np.load(load_filename+'_endstep_data_mean_dict.npy', allow_pickle=True).item()
        yaxis_std_dict = np.load(load_filename+'_endstep_data_std_dict.npy', allow_pickle=True).item()
    except (FileNotFoundError):
        load_filename = 'pt_localization_excitation_L={0}_maxN={1}_system_majorana_mapping_disorder_everystep=True_{2}'.format(str(sys_size),str(max_N),model_name)
        yaxis_dict = np.load(load_filename+'_endstep_data_mean_dict.npy', allow_pickle=True).item()
        yaxis_std_dict = np.load(load_filename+'_endstep_data_std_dict.npy', allow_pickle=True).item()
    fig, ax = plt.subplots(figsize=(10, 6))
    #ax.set_ylabel(r'Excitations $N_{\rm ex}$',fontsize=FONTSIZE_AXIS)
    ax.set_xlabel(r'$\sigma$ disorder',fontsize=FONTSIZE_AXIS)
    
    # Set log scale
    ax.set_yscale('log')
    ax.set_xscale('log')  

    plt.tight_layout()
    # TODO: Read the correct files and remove the 0 STATIC disorder
    xaxis = []#np.array([k[1] for k in yaxis_dict.keys()])
    yaxis = []
    yerr_arr = []

    # Get the every step noise
    k_every_step_val = sorted(yaxis_dict.keys())[0][0]

    for k in sorted(yaxis_dict.keys()):
        if k == (k_every_step_val,0):
            continue
        #yerr_arr = yaxis_std_dict[k]
        #l_yaxis = []
        #l_yerr_arr = []
        #for n_opt in xaxis:
        #    l_yaxis.append(yaxis[n_opt])
        #    l_yerr_arr.append(yerr_arr[n_opt])

        max_N_to_plot = max(yaxis_dict[k].keys())
        #print(yaxis_dict[k])
        
        # Ignore negative values
        val_y = yaxis_dict[k][max_N_to_plot]-yaxis_dict[(k_every_step_val,0)][max_N_to_plot]
        if val_y <0:
            continue

        xaxis.append(k[1])
        yaxis.append(yaxis_dict[k][max_N_to_plot]-yaxis_dict[(k_every_step_val,0)][max_N_to_plot])
        yerr_arr.append(yaxis_std_dict[k][max_N_to_plot])
    print(xaxis)
    print(yaxis)
    print(yerr_arr)
    ax.errorbar(np.array(xaxis),np.array(yaxis),ls='--',marker='o',markersize=4,capsize=2,yerr=yerr_arr,label=r'$L='+str(sys_size)+'$')
    fit_params = scipy.optimize.curve_fit(lambda x,a,b: a+b*x, np.log(np.array(xaxis)), np.log(np.array(yaxis)))
    print(fit_params[0])
    fit_slope = fit_params[0][1]
    #print('yaxis:'+str(np.log(np.array(yaxis[:])[:])))
    print('L={0}= {1}'.format(str(sys_size),str(fit_slope)))
    ax.plot(np.array(xaxis),np.exp(fit_params[0][0]+fit_params[0][1]*np.log(np.array(xaxis))),color='tab:red',ls='-',label=r'Linear fit, $f(\sigma)='+str(fit_slope)+r'\sigma+'+str(fit_params[0][0])+r'$')
    #print('yaxis:'+str(np.log(np.array(yaxis[:])[:])))
    #print('sigma={0} with slope={1}'.format(str(k), str(fit_slope)))
    
    ax.legend(loc='lower right',fontsize=15)
    ax.figure.savefig(load_filename+'.png',dpi=300)
    ax.figure.savefig(load_filename+'.svg')

In [ ]:
# # Copy scaling code from the other file to here
# # Plot the saturation values as a function of the disorder
# def plot_sat_values_with_fixed_slope(sys_size,max_N,rad=1,trot=False,match_guess=False):
#     # Load data from files
#     try:
#         load_filename = 'pt_localization_excitation_L={0}_maxN={1}_system_majorana_mapping_disorder_everystep=True_rad={2}_trotter={3}_{4}'.format(str(sys_size),str(max_N),str(rad),str(trot),model_name)
#         yaxis_dict = np.load(load_filename+'_endstep_data_mean_dict.npy', allow_pickle=True).item()
#         yaxis_std_dict = np.load(load_filename+'_endstep_data_std_dict.npy', allow_pickle=True).item()
#     except (FileNotFoundError):
#         load_filename = 'pt_localization_excitation_L={0}_maxN={1}_system_majorana_mapping_disorder_everystep=True_{2}'.format(str(sys_size),str(max_N),model_name)
#         yaxis_dict = np.load(load_filename+'_endstep_data_mean_dict.npy', allow_pickle=True).item()
#         yaxis_std_dict = np.load(load_filename+'_endstep_data_std_dict.npy', allow_pickle=True).item()
#     fig, ax = plt.subplots(figsize=(10, 6))
#     #ax.set_ylabel(r'Excitations $N_{\rm ex}$',fontsize=FONTSIZE_AXIS)
#     ax.set_xlabel(r'$\sigma$ disorder',fontsize=FONTSIZE_AXIS)
    
#     # Set log scale
#     ax.set_yscale('log')
#     ax.set_xscale('log')  

#     plt.tight_layout()
#     # TODO: Read the correct files and remove the 0 STATIC disorder
#     xaxis = []#np.array([k[1] for k in yaxis_dict.keys()])
#     yaxis = []
#     yerr_arr = []

#     # Get the every step noise
#     k_every_step_val = sorted(yaxis_dict.keys())[0][0]

#     for k in sorted(yaxis_dict.keys()):
#         if k == (k_every_step_val,0):
#             continue
#         #yerr_arr = yaxis_std_dict[k]
#         #l_yaxis = []
#         #l_yerr_arr = []
#         #for n_opt in xaxis:
#         #    l_yaxis.append(yaxis[n_opt])
#         #    l_yerr_arr.append(yerr_arr[n_opt])

#         max_N_to_plot = max(yaxis_dict[k].keys())
#         #print(yaxis_dict[k])
        
#         # Ignore negative values
#         val_y = yaxis_dict[k][max_N_to_plot]-yaxis_dict[(k_every_step_val,0)][max_N_to_plot]
#         if val_y <0:
#             continue

#         xaxis.append(k[1])
#         yaxis.append(yaxis_dict[k][max_N_to_plot]-yaxis_dict[(k_every_step_val,0)][max_N_to_plot])
#         yerr_arr.append(yaxis_std_dict[k][max_N_to_plot])
#     print(xaxis)
#     print(yaxis)
#     print(yerr_arr)
#     ax.errorbar(np.array(xaxis),np.array(yaxis),ls='--',marker='o',markersize=4,capsize=2,yerr=yerr_arr,label=r'$L='+str(sys_size)+'$')
    
#     if match_guess:
#         guess_co = np.log(0.36)
#         ax.plot(np.array(xaxis),np.exp(guess_co+2*np.log(np.array(xaxis))),color='tab:red',ls='-',label=r'Linear fit, $f(\sigma)='+str(2)+r'\sigma+'+str(guess_co)+r'$')
#         load_filename += '_guess_coefficient=0.36'
#     else:
#         fit_params = scipy.optimize.curve_fit(lambda x,a: a+2*x, np.log(np.array(xaxis)), np.log(np.array(yaxis)))
#         print(fit_params[0])
#         ax.plot(np.array(xaxis),np.exp(fit_params[0][0]+2*np.log(np.array(xaxis))),color='tab:red',ls='-',label=r'Linear fit, $f(\sigma)='+str(2)+r'\sigma+'+str(fit_params[0][0])+r'$')
#     #print('yaxis:'+str(np.log(np.array(yaxis[:])[:])))
#     #print('sigma={0} with slope={1}'.format(str(k), str(fit_slope)))
    
#     ax.legend(loc='lower right',fontsize=15)
#     ax.figure.savefig(load_filename+'_with_fixed_slope.png',dpi=300)
#     ax.figure.savefig(load_filename+'_with_fixed_slope.svg')

In [ ]:
# Define variables
disorder_xx_var_arr = [0]#[0.1]
static_disorder_xx_var_arr = sorted(np.append(2e-3*np.array(range(10,200,10)),0))
max_N = 1000#30
N_start = 1
radi = 1
to_trotter_radius = False
N_opt = [max_N]#list(range(N_start,max_N+1,5))
L_sizes = [30]
disorder_count = 200
is_disorder_every_step = False
model_name = 'static_disorder_with_everystep='+str(disorder_xx_var_arr[0])+'_for_scaling_only_end_step'
print(N_opt)
print(static_disorder_xx_var_arr)
print(model_name)

In [ ]:
save_disorder_data(L_sizes,N_opt,disorder_xx_var_arr,static_disorder_xx_var_arr,cur_disorder_count=disorder_count,_model_name=model_name,radiu=radi,to_trotter= to_trotter_radius)

In [ ]:
# Plot the Excitations
for cur_sys_size in L_sizes:
    # TODO: save them with actual names
    plot_kz_scaling_disorder_static_new_format(N_opt,cur_sys_size,N_opt[-1],model_name,rad=radi,trot=to_trotter_radius)

In [ ]:
# Plot the saturation N_max values
#for cur_sys_size in L_sizes:
#    plot_sat_values(cur_sys_size,max_N)
# Plot the saturation N_max values
for cur_sys_size in L_sizes:
    plot_sat_values_with_fixed_slope(cur_sys_size,max_N)
for cur_sys_size in L_sizes:
    plot_sat_values_with_fixed_slope(cur_sys_size,max_N,match_guess=True)

## Merge results (AVOID USAGE)

In [ ]:
# Merge results of disorders for system size L and save it
# TODO: Chnage it to a function for quick merging
sys_size = 50
model1 = 'nisq_parameters_low_nsteps'
model2 = 'nisq_parameters'
maxN1 = 28
maxN2 = 1000
load_filename1 = 'pt_localization_excitation_L={0}_maxN={1}_system_majorana_mapping_disorder_everystep=True_{2}'.format(str(sys_size),str(maxN1),model1)
load_filename2 = 'pt_localization_excitation_L={0}_maxN={1}_system_majorana_mapping_disorder_everystep=True_{2}'.format(str(sys_size),str(maxN2),model2)
yaxis_dict1 = np.load(load_filename1+'_endstep_data_mean_dict.npy', allow_pickle=True).item()
yaxis_std_dict1 = np.load(load_filename1+'_endstep_data_std_dict.npy', allow_pickle=True).item()
yaxis_dict2 = np.load(load_filename2+'_endstep_data_mean_dict.npy', allow_pickle=True).item()
yaxis_std_dict2 = np.load(load_filename2+'_endstep_data_std_dict.npy', allow_pickle=True).item()
print(yaxis_std_dict1)
print(yaxis_std_dict2)
yaxis_res = dict()
for k in yaxis_dict1.keys() & yaxis_dict2.keys():
    temp1 = yaxis_dict1.get(k)
    temp2 = yaxis_dict2.get(k)
    temp1.update(temp2)
    yaxis_res[k] = temp1
yaxis_std_res = dict()
for k in yaxis_std_dict1.keys() & yaxis_std_dict2.keys():
    temp1 = yaxis_std_dict1.get(k)
    temp2 = yaxis_std_dict2.get(k)
    temp1.update(temp2)
    yaxis_std_res[k] = temp1
print(yaxis_std_res)
result_name = 'merged_'+model2
res_nmax = max(maxN1,maxN2)
result_filename = 'pt_localization_excitation_L={0}_maxN={1}_system_majorana_mapping_disorder_everystep=True_{2}'.format(str(sys_size),str(res_nmax),result_name)
np.save(result_filename+'_endstep_data_mean_dict.npy',yaxis_res)
np.save(result_filename+'_endstep_data_std_dict.npy',yaxis_std_res)

# Reworked plots with the new $N_{steps}$ definition UPDATE FROM 27.03.22

## Plot definitions (from FloquetKibbleZurek.ipynb)

In [ ]:
# Font definitions
FONTSIZE_LEGEND = 22
FONTSIZE_AXIS = 24
FONTSIZE_TITLE = 24
FONTSIZE_TICKS = 20

# OLD VERSION WITH OLD FILENAMES - DO NOT USE
#def plot_kz_scaling_disorder_deprecated(sys_size):
#    sigmas=[]
#    defects=[]
#    defects_err=[]
#
#    #plt.tight_layout()
#
#    # Load data from files
#    old_filename = 'kz_scaling_{0}system_fermion_mapping_disorder_everystep=True_optimized_new_version'.format(str(sys_size))
#    yaxis_dict = np.load(old_filename+'_data_mean_dict.npy', allow_pickle=True).item()
#    yaxis_std_dict = np.load(old_filename+'_data_std_dict.npy', allow_pickle=True).item()
#    
#    for k,yaxis in sorted(yaxis_dict.items()):
#        sigmas.append(k)
#        defects.append(yaxis[1:])
#        defects_err.append(yaxis_std_dict[k][1:])
#    
#    #ax.figure.savefig(old_filename+'.png')
#    #ax.figure.savefig(old_filename+'.svg')
#    return sigmas,defects,defects_err

# New data format
def data_kz_scaling_disorder(sys_size,max_N,model_name,xaxis=None):
    # Load data from files
    load_filename = 'pt_localization_excitation_L={0}_maxN={1}_system_majorana_mapping_disorder_everystep=True_rad=1_trotter=False_{2}'.format(str(sys_size),str(max_N),model_name)
    yaxis_dict = np.load(load_filename+'_endstep_new_nsteps_definition_corrected_noise_data_mean_dict.npy', allow_pickle=True).item()
    yaxis_std_dict = np.load(load_filename+'_endstep_new_nsteps_definition_corrected_noise_data_std_dict.npy', allow_pickle=True).item()

    # Choose the xaxis to be the N's of the first in the dictionary
    if xaxis is None:
        xaxis = list(list(yaxis_dict.values())[0].keys())
    
    sigmas = []
    defects = []
    defects_err = []
    for k,yaxis in sorted(yaxis_dict.items()):
        sigmas.append(k)
        l_yaxis = []
        l_yerr_arr = []
        for n_opt in xaxis:
            l_yaxis.append(yaxis[n_opt])
            l_yerr_arr.append(yaxis_std_dict[k][n_opt])
        defects.append(np.array(l_yaxis))
        #defects.append(np.array(l_yaxis)*(sys_size-1)/(sys_size)+(1/(2*sys_size)))
        #defects.append(np.array(l_yaxis)+(1/(2*sys_size)))
        defects_err.append(np.array(l_yerr_arr))
    
    return np.array(xaxis),np.array(sigmas),np.array(defects),np.array(defects_err)

In [ ]:
def plot_zero_disorder_match(L_size_arr,max_N,model_name,xaxis=None,add1overL=False,fit_region=[]):
    # Initialize plot
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.set_ylabel(r'$d_\mathrm{ideal}$',fontsize=FONTSIZE_AXIS)
    ax.set_xlabel(r'$N$',fontsize=FONTSIZE_AXIS)

    # Set log scale
    ax.set_yscale('log')
    ax.set_xscale('log')

    is_first=True
    # Change tick sizes
    ax.tick_params(axis = 'both', which = 'major', labelsize = FONTSIZE_TICKS)
    for L_size in L_size_arr:
        if L_size<10:
            continue
        xaxis,sigmas,defects,defects_err=data_kz_scaling_disorder(L_size,max_N,model_name,xaxis)
        if is_first:
            #ax.set_xticks(xaxis) # NO MINOR BOOL FOR NEWER MATPLOTLIB
            is_first=False
        for s in range(len(sigmas)):
            cur_disorder = sigmas[s]
            defects_corrected=np.array(defects[s])
            ax.errorbar(xaxis,defects_corrected,ls='--',label=r'$L='+str(L_size)+r'$',marker='o',markersize=4,capsize=2,yerr=defects_err[s])
        if max(L_size_arr)==L_size:
            if (len(fit_region)<2) : fit_region=range(0,len(xaxis))
            # This is the old fit that tries to fit also the expoennt b
            #fit_params = scipy.optimize.curve_fit(lambda x,a,b: a+b*x, np.log(xaxis[fit_region]), np.log(defects_corrected[fit_region]))
            #a = fit_params[0][0]
            #b = fit_params[0][1]
            #print("For L_size={0} we have params={1}".format(str(L_size),str((a,b))))
            #ax.plot(xaxis,np.exp(a+b*np.log(xaxis)),c='r',ls='--',label = r'${0}N^{{{1}}}$'.format(str(np.exp(a))[0:5],str(b)[0:5]))
            # Here we try to fit only the coefficient a for the -0.5 exponent  
            fit_params = scipy.optimize.curve_fit(lambda x,a: a*x**(-0.5), xaxis[fit_region[0]:fit_region[1]], np.array(defects[s])[fit_region[0]:fit_region[1]])#, sigma=np.array(defects_err[s])[fit_region])
            a = fit_params[0][0]
            err = np.sqrt(fit_params[1][0][0])
            #percent_error = max(np.exp(err)-1,1-np.exp(-err))
            print("Prefactor:{0}".format(a))
            #print("Percent error:{0}".format(percent_error))
            print("{0}---{1}".format(a-err,a+err))
            # See also https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html:
            # To compute one standard deviation errors on the parameters use perr = np.sqrt(np.diag(pcov)).
            print('standard deviation={0}'.format(str(err)))
            ax.plot(xaxis,a*np.exp(-0.5*np.log(xaxis)),c='dimgray',lw=4,ls='--',label = r'$\mathrm{KZ}$')#r'${0:.3f}N^{{-0.5}}$'.format(a))
    ax.legend(loc='lower left',fontsize=FONTSIZE_LEGEND)
    # UPDATE 27.03.22: We now plot into pdf instead of png or svg
    name = 'figures/kz_scaling_{0}system_maxN={1}_fermion_mapping_disorder_everystep=True_new_nsteps_definition_optimized_new_version'.format(str(L_size_arr),max_N)
    ax.figure.savefig(name+'.pdf', bbox_inches='tight')
    plt.show()
    return ax

In [ ]:
# Plot the saturation values as a function of the disorder for static disorder
def plot_sat_values_with_fixed_slope(sys_size_arr,inp_max_N,rad=1,trot=False,match_guess=False,apply_opacity_error=False):
    
    # Initialize the figure and axes
    fig, ax = plt.subplots(figsize=(10, 6))
    #ax.set_ylabel(r'Excitations $N_{\rm ex}$',fontsize=FONTSIZE_AXIS)
    ax.set_ylabel(r'$d_\mathrm{disorder} = d-d_\mathrm{ideal}$',fontsize=FONTSIZE_AXIS)
    ax.set_xlabel(r'$\sigma_\mathrm{disorder}$',fontsize=FONTSIZE_AXIS)
    
    # Set log scale
    ax.set_yscale('log')
    ax.set_xscale('log')  

    plt.tight_layout()

    # UPDATE 27.03.22: Force match_guess
    #if not match_guess:
    #    raise Exception("we must match_guess=True as we plot here label of 0.8\sigma^2")

    is_first = True
    for sys_size in sys_size_arr:
        
            # Load data from files
            try:
                load_filename = 'pt_localization_excitation_L={0}_maxN={1}_system_majorana_mapping_disorder_everystep=True_rad={2}_trotter={3}_{4}'.format(str(sys_size),str(inp_max_N),str(rad),str(trot),model_name)
                yaxis_dict = np.load(load_filename+'_endstep_new_nsteps_definition_corrected_noise_data_mean_dict.npy', allow_pickle=True).item()
                yaxis_std_dict = np.load(load_filename+'_endstep_new_nsteps_definition_corrected_noise_data_std_dict.npy', allow_pickle=True).item()
            except (FileNotFoundError):
                load_filename = 'pt_localization_excitation_L={0}_maxN={1}_system_majorana_mapping_disorder_everystep=True_{2}'.format(str(sys_size),str(inp_max_N),model_name)
                yaxis_dict = np.load(load_filename+'_endstep_new_nsteps_definition_corrected_noise_data_mean_dict.npy', allow_pickle=True).item()
                yaxis_std_dict = np.load(load_filename+'_endstep_new_nsteps_definition_corrected_noise_data_std_dict.npy', allow_pickle=True).item()
            max_N_arr = list(yaxis_dict[0,0].keys())
            for cur_max_N in max_N_arr:
                # TODO: Read the correct files and remove the 0 STATIC disorder
                xaxis = []#np.array([k[1] for k in yaxis_dict.keys()])
                yaxis = []
                yerr_arr = []

                # Get the every step noise
                k_every_step_val = sorted(yaxis_dict.keys())[0][0]

                for k in sorted(yaxis_dict.keys()):
                    if k == (k_every_step_val,0):
                        continue
                    
                    # Ignore negative values
                    val_y = yaxis_dict[k][cur_max_N]-yaxis_dict[(k_every_step_val,0)][cur_max_N]
                    if val_y <0:
                        continue

                    xaxis.append(k[1])
                    yaxis.append(yaxis_dict[k][cur_max_N]-yaxis_dict[(k_every_step_val,0)][cur_max_N])
                    yerr_arr.append(yaxis_std_dict[k][cur_max_N])
                print(xaxis)
                print(yaxis)
                print(yerr_arr)
                if not apply_opacity_error:
                    ax.errorbar(np.array(xaxis),np.array(yaxis),ls='--',marker='o',markersize=6,capsize=5,lw=3,yerr=yerr_arr,label=r'$L='+str(sys_size)+r', N = '+str(cur_max_N)+r'$')
                else:
                    markers, caps, bars = ax.errorbar(np.array(xaxis),np.array(yaxis),ls='--',marker='o',markersize=6,capsize=5,lw=3,yerr=yerr_arr,label=r'$L='+str(sys_size)+r', N = '+str(cur_max_N)+r'$')
                    [bar.set_alpha(0.35) for bar in bars]
                    [cap.set_alpha(0.35) for cap in caps]
                # Fit for the maximum length
                if ((sys_size==30 and cur_max_N==20) or (len(sys_size_arr)>1 and sys_size == sys_size_arr[2])) and is_first:
                    is_first=False
                    if match_guess:
                        guess_coe = 1
                        guess_co = np.log(guess_coe)
                        ax.plot(np.array(xaxis),np.exp(guess_co+2*np.log(np.array(xaxis))),color='aquamarine',zorder=-10,ls='-',lw=10,label=r'$f(\sigma_\mathrm{disorder})=\sigma_\mathrm{disorder}^2$')
                    else:
                        print("START FOR FOR N={0}".format(str(cur_max_N)))
                        fit_params = scipy.optimize.curve_fit(lambda x,a: a*x**2, np.array(xaxis), np.array(yaxis),sigma=np.array(yerr_arr))
                        print(fit_params)
                        a = fit_params[0][0]
                        err = np.sqrt(np.diag(fit_params[1]))[0]
                        #percent_error = max(np.exp(err)-1,1-np.exp(-err))
                        #print(percent_error)
                        print("Prefactor={0}".format(str(a)))
                        print("{0}---{1}".format(a-err,a+err))
                        # See also https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html:
                        # To compute one standard deviation errors on the parameters use perr = np.sqrt(np.diag(pcov)).
                        print('standard deviation={0}'.format(str(err)))
                        # The standard deviation of the parameters are the square roots of the diagonal elements of pcov=fit_params[1]. See https://stackoverflow.com/a/21844726
                        ax.plot(np.array(xaxis),a*np.exp(2*np.log(np.array(xaxis))),color='aquamarine',zorder=-10,ls='-',lw=10,label=r'$f(\sigma_\mathrm{{disorder}})={0:.2f}\sigma_\mathrm{{disorder}}^2$'.format(a))
    load_filename = 'pt_localization_excitation_L={0}_maxN={1}_system_majorana_mapping_disorder_everystep=True_new_nsteps_definition_corrected_noise_rad={2}_trotter={3}_{4}'.format(str(sys_size_arr),str(max_N_arr),str(rad),str(trot),model_name[:5])
    if match_guess:
        load_filename += '_coe=1'
    
    # Change tick sizes
    ax.tick_params(axis = 'both', which = 'major', labelsize = FONTSIZE_TICKS)
    
    ax.legend(loc='lower right',fontsize=FONTSIZE_LEGEND)
    ax.figure.savefig(load_filename+'.pdf', bbox_inches='tight')

In [ ]:
# Plot the saturation values as a function of the disorder for static disorder
def plot_sat_values_with_fixed_slope_noise(sys_size_arr,inp_max_N,rad=1,trot=False,match_guess=False):
    
    # Initialize the figure and axes
    fig, ax = plt.subplots(figsize=(10, 6))
    #ax.set_ylabel(r'Excitations $N_{\rm ex}$',fontsize=FONTSIZE_AXIS)
    ax.set_ylabel(r'$d_\mathrm{noise} = d-d_\mathrm{ideal}$',fontsize=FONTSIZE_AXIS)
    ax.set_xlabel(r'$\sigma_\mathrm{noise}$',fontsize=FONTSIZE_AXIS)
    
    # Set log scale
    ax.set_yscale('log')
    ax.set_xscale('log')  

    plt.tight_layout()

    # UPDATE 27.03.22: Force match_guess
    #if not match_guess:
    #    raise Exception("we must match_guess=True as we plot here label of 0.8\sigma^2")

    is_first = True
    for sys_size in sys_size_arr:
        
            # Load data from files
            try:
                load_filename = 'pt_localization_excitation_L={0}_maxN={1}_system_majorana_mapping_disorder_everystep=True_rad={2}_trotter={3}_{4}'.format(str(sys_size),str(inp_max_N),str(rad),str(trot),model_name)
                yaxis_dict = np.load(load_filename+'_endstep_new_nsteps_definition_corrected_noise_data_mean_dict.npy', allow_pickle=True).item()
                yaxis_std_dict = np.load(load_filename+'_endstep_new_nsteps_definition_corrected_noise_data_std_dict.npy', allow_pickle=True).item()
            except (FileNotFoundError):
                load_filename = 'pt_localization_excitation_L={0}_maxN={1}_system_majorana_mapping_disorder_everystep=True_{2}'.format(str(sys_size),str(inp_max_N),model_name)
                yaxis_dict = np.load(load_filename+'_endstep_new_nsteps_definition_corrected_noise_data_mean_dict.npy', allow_pickle=True).item()
                yaxis_std_dict = np.load(load_filename+'_endstep_new_nsteps_definition_corrected_noise_data_std_dict.npy', allow_pickle=True).item()
            max_N_arr = list(yaxis_dict[0,0].keys())
            for cur_max_N in max_N_arr:
                # TODO: Read the correct files and remove the 0 STATIC disorder
                xaxis = []#np.array([k[1] for k in yaxis_dict.keys()])
                yaxis = []
                yerr_arr = []

                # Get the static disorder
                k_dis_val = sorted(yaxis_dict.keys())[0][1]
                print(k_dis_val)

                for k in sorted(yaxis_dict.keys()):
                    if k == (0,k_dis_val):
                        continue
                    
                    # Ignore negative values
                    val_y = yaxis_dict[k][cur_max_N]-yaxis_dict[(0,k_dis_val)][cur_max_N]
                    if val_y <0:
                        continue

                    xaxis.append(k[0])
                    yaxis.append(yaxis_dict[k][cur_max_N]-yaxis_dict[(0,k_dis_val)][cur_max_N])
                    yerr_arr.append(yaxis_std_dict[k][cur_max_N])
                print(xaxis)
                print(yaxis)
                print(yerr_arr)
                ax.errorbar(np.array(xaxis),np.array(yaxis),ls='--',marker='o',markersize=6,capsize=5,lw=3,yerr=yerr_arr,label=r'$L='+str(sys_size)+r', N = '+str(cur_max_N)+r'$')
                
                if is_first and sys_size==max(sys_size_arr):
                    is_first=False
                    if match_guess:
                        guess_coe = 0.8
                        guess_co = np.log(guess_coe)
                        ax.plot(np.array(xaxis),np.exp(guess_co+2*np.log(np.array(xaxis))),color='aquamarine',zorder=-10,ls='-',lw=10,label=r'$f(\sigma)=0.8\sigma^2$')
                    else:
                        fit_params = scipy.optimize.curve_fit(lambda x,a: a*cur_max_N*x**2, np.array(xaxis), np.array(yaxis),sigma=np.array(yerr_arr))
                        a = fit_params[0][0]
                        err = np.sqrt(fit_params[1][0][0])
                        #percent_error = max(np.exp(err)-1,1-np.exp(-err))
                        #print(percent_error)
                        print("Prefactor={0}".format(str(a)))
                        print("{0}---{1}".format(a-err,a+err))
                        # See also https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html:
                        # To compute one standard deviation errors on the parameters use perr = np.sqrt(np.diag(pcov)).
                        print('standard deviation={0}'.format(str(err)))
                        print(fit_params[0])

                        ax.plot(np.array(xaxis),a*cur_max_N*np.exp(2*np.log(np.array(xaxis))),color='aquamarine',zorder=-10,ls='-',lw=10,label=r'$f(\sigma_\mathrm{{noise}})={0:.2f}N\sigma_\mathrm{{noise}}^2$'.format(a))

                        # UPDATE 06.08.22: Without fit
                        #ax.plot(np.array(xaxis),2*cur_max_N*np.exp(2*np.log(np.array(xaxis))),color='aquamarine',zorder=-10,ls='-',lw=10,label=r'$f(\sigma_\mathrm{noise})=2 N\sigma_\mathrm{noise}^2$')
    load_filename = 'pt_localization_excitation_L={0}_maxN={1}_system_majorana_mapping_disorder_everystep=True_new_nsteps_definition_corrected_noise_rad={2}_trotter={3}_{4}'.format(str(sys_size_arr),str(max_N_arr),str(rad),str(trot),model_name[:5])     
    if match_guess:
        load_filename += '_coe='+str(guess_coe)
    
    # Change tick sizes
    ax.tick_params(axis = 'both', which = 'major', labelsize = FONTSIZE_TICKS)
    
    ax.legend(loc='lower right',fontsize=FONTSIZE_LEGEND)
    ax.figure.savefig(load_filename+'.pdf', bbox_inches='tight')

In [ ]:
# Plot the saturation values as a function of the noise (every step)
def plot_sat_values_with_fixed_slope_noise_nmax(sys_size,cur_nmax,rad=1,trot=False,match_guess=False,cap_nmax=0):
    # Initialize the figure and axes
    fig, ax = plt.subplots(figsize=(10, 6))
    #ax.set_ylabel(r'Excitations $N_{\rm ex}$',fontsize=FONTSIZE_AXIS)
    ax.set_ylabel(r'$d_\mathrm{noise} = d-d_\mathrm{ideal}$',fontsize=FONTSIZE_AXIS)
    ax.set_xlabel(r'$N$',fontsize=FONTSIZE_AXIS)
    
    # Set log scale
    ax.set_yscale('log')
    ax.set_xscale('log')

    plt.tight_layout()

    # UPDATE 27.03.22: Force match_guess
    #if not match_guess:
    #    raise Exception("we must match_guess=True as we plot here label of 0.8\sigma^2")

    is_first = True
    try:
        load_filename = 'pt_localization_excitation_L={0}_maxN={1}_system_majorana_mapping_disorder_everystep=True_rad={2}_trotter={3}_{4}'.format(str(sys_size),str(cur_nmax),str(rad),str(trot),model_name)
        yaxis_dict = np.load(load_filename+'_endstep_new_nsteps_definition_corrected_noise_data_mean_dict.npy', allow_pickle=True).item()
        yaxis_std_dict = np.load(load_filename+'_endstep_new_nsteps_definition_corrected_noise_data_std_dict.npy', allow_pickle=True).item()
    except (FileNotFoundError):
        load_filename = 'pt_localization_excitation_L={0}_maxN={1}_system_majorana_mapping_disorder_everystep=True_{2}'.format(str(sys_size),str(cur_nmax),model_name)
        yaxis_dict = np.load(load_filename+'_endstep_new_nsteps_definition_corrected_noise_data_mean_dict.npy', allow_pickle=True).item()
        yaxis_std_dict = np.load(load_filename+'_endstep_new_nsteps_definition_corrected_noise_data_std_dict.npy', allow_pickle=True).item()
    max_N_arr = list(yaxis_dict[0,0].keys())
    # Get the static disorder
    k_dis_val = sorted(yaxis_dict.keys())[0][1]
    print(max_N_arr)

    val_fit = None
    for k in sorted(yaxis_dict.keys(),reverse=True):
        if k == (0,k_dis_val):
            continue
        # TODO: Read the correct files and remove the 0 STATIC disorder
        xaxis = []#np.array([k[1] for k in yaxis_dict.keys()])
        yaxis = []
        yerr_arr = []
        for cur_max_N in max_N_arr:
            if cur_max_N < cap_nmax:
                continue
            # Ignore negative values
            val_y = yaxis_dict[k][cur_max_N]-yaxis_dict[(0,k_dis_val)][cur_max_N]
            if val_y <0:
                continue
            
            xaxis.append(cur_max_N)
            yaxis.append(yaxis_dict[k][cur_max_N]-yaxis_dict[(0,k_dis_val)][cur_max_N])
            yerr_arr.append(yaxis_std_dict[k][cur_max_N])
        #print(xaxis)
        #print(yaxis)
        #print(yerr_arr)
        ax.errorbar(np.array(xaxis),np.array(yaxis),ls='--',marker='o',markersize=6,capsize=5,lw=3,yerr=yerr_arr,label=r'$L='+str(sys_size)+r'$, $\sigma_\mathrm{noise} = '+str(k[0])+r'$')
        
        cur_sigma = k[0]
        if cur_sigma>=1e-3 and is_first:
            is_first=False
            #if match_guess:
            #    guess_coe = 0.8
            #   guess_co = np.log(guess_coe)
            #    ax.plot(np.array(xaxis),np.exp(guess_co+2*np.log(np.array(xaxis))),color='aquamarine',zorder=-10,ls='-',lw=10,label=r'$f(\sigma)=0.8\sigma^2$')
            #else:
            # the sigma of the noise
            
            fit_params = scipy.optimize.curve_fit(lambda x,a: a*x*cur_sigma**2, np.array(xaxis), np.array(yaxis),sigma=np.array(yerr_arr))
            print("Sigma to fit:{0}".format(cur_sigma))
            a = fit_params[0][0]
            err = np.sqrt(fit_params[1][0][0])
            #percent_error = max(np.exp(err)-1,1-np.exp(-err))
            #print(percent_error)
            print("Prefactor={0}".format(str(a)))
            print("{0}---{1}".format(a-err,a+err))
            # See also https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html:
            # To compute one standard deviation errors on the parameters use perr = np.sqrt(np.diag(pcov)).
            print('standard deviation={0}'.format(str(err)))

            val_fit = a
            ax.plot(np.array(xaxis),a*np.array(xaxis)*cur_sigma**2,color='aquamarine',zorder=-10,ls='-',lw=10,label=r'$f(N)={0:.2f}N\sigma_\mathrm{{noise}}^2$'.format(a))
        # Adding aqua marine marker lines to all other lines
        else:
            cur_sigma = k[0]
            ax.plot(np.array(xaxis),val_fit*cur_sigma**2*np.array(xaxis),color='aquamarine',zorder=-10,ls='-',lw=10)
    load_filename = 'pt_localization_excitation_L={0}_maxN={1}_system_majorana_mapping_disorder_everystep=True_new_nsteps_definition_corrected_noise_rad={2}_trotter={3}_{4}'.format(str(sys_size),str(cur_nmax),str(rad),str(trot),model_name[:5])

    # Change tick sizes
    ax.tick_params(axis = 'both', which = 'major', labelsize = FONTSIZE_TICKS)
    ax.legend(loc='lower right',fontsize=FONTSIZE_LEGEND)
    ax.figure.savefig(load_filename+'.pdf', bbox_inches='tight')

In [ ]:
# Plot the saturation values as a function of the disorder for static disorder
def plot_sat_values_with_fixed_slope_noise_alternative(sys_size_arr,inp_max_N,rad=1,trot=False,match_guess=False):
    
    # Initialize the figure and axes
    fig, ax = plt.subplots(figsize=(10, 6))
    #ax.set_ylabel(r'Excitations $N_{\rm ex}$',fontsize=FONTSIZE_AXIS)
    ax.set_ylabel(r'$d_\mathrm{noise} = d-d_\mathrm{ideal}$',fontsize=FONTSIZE_AXIS)
    ax.set_xlabel(r'$\sigma_\mathrm{noise}$',fontsize=FONTSIZE_AXIS)
    
    # Set log scale
    ax.set_yscale('log')
    ax.set_xscale('log')  

    plt.tight_layout()

    # UPDATE 27.03.22: Force match_guess
    #if not match_guess:
    #    raise Exception("we must match_guess=True as we plot here label of 0.8\sigma^2")

    is_first = True
    for sys_size in sys_size_arr:
        
            # Load data from files
            try:
                load_filename = 'pt_localization_excitation_L={0}_maxN={1}_system_majorana_mapping_disorder_everystep=True_rad={2}_trotter={3}_{4}'.format(str(sys_size),str(inp_max_N),str(rad),str(trot),model_name)
                yaxis_dict = np.load(load_filename+'_endstep_new_nsteps_definition_corrected_noise_data_mean_dict.npy', allow_pickle=True).item()
                yaxis_std_dict = np.load(load_filename+'_endstep_new_nsteps_definition_corrected_noise_data_std_dict.npy', allow_pickle=True).item()
            except (FileNotFoundError):
                load_filename = 'pt_localization_excitation_L={0}_maxN={1}_system_majorana_mapping_disorder_everystep=True_{2}'.format(str(sys_size),str(inp_max_N),model_name)
                yaxis_dict = np.load(load_filename+'_endstep_new_nsteps_definition_corrected_noise_data_mean_dict.npy', allow_pickle=True).item()
                yaxis_std_dict = np.load(load_filename+'_endstep_new_nsteps_definition_corrected_noise_data_std_dict.npy', allow_pickle=True).item()
            max_N_arr = list(yaxis_dict[0,0].keys())
            for cur_max_N in max_N_arr:
                # TODO: Read the correct files and remove the 0 STATIC disorder
                xaxis = []#np.array([k[1] for k in yaxis_dict.keys()])
                yaxis = []
                yerr_arr = []

                # Get the static disorder
                k_dis_val = sorted(yaxis_dict.keys())[0][1]
                print(k_dis_val)

                for k in sorted(yaxis_dict.keys()):
                    if k == (0,k_dis_val):
                        continue
                    
                    # Ignore negative values
                    val_y = yaxis_dict[k][cur_max_N]-yaxis_dict[(0,k_dis_val)][cur_max_N]
                    if val_y <0:
                        continue

                    xaxis.append(k[0])
                    yaxis.append(yaxis_dict[k][cur_max_N]-yaxis_dict[(0,k_dis_val)][cur_max_N])
                    yerr_arr.append(yaxis_std_dict[k][cur_max_N])
                print(xaxis)
                print(yaxis)
                print(yerr_arr)
                ax.errorbar(np.array(xaxis),np.array(yaxis),ls='--',marker='o',markersize=6,capsize=5,lw=3,yerr=yerr_arr,label=r'$L='+str(sys_size)+r', N = '+str(cur_max_N)+r'$')
                
                if is_first:
                    is_first=False
                    #if match_guess:
                    #    guess_coe = 0.8
                    #    guess_co = np.log(guess_coe)
                    #    ax.plot(np.array(xaxis),np.exp(guess_co+2*np.log(np.array(xaxis))),color='aquamarine',zorder=-10,ls='-',lw=10,label=r'$f(\sigma)=0.8\sigma^2$')
                    #else:

                    # UPDATE 08.06.2022 - DON'T FIT
                    #fit_params = scipy.optimize.curve_fit(lambda x,a: a+2*x, np.log(np.array(xaxis)), np.log(np.array(yaxis)))
                    #print(fit_params[0])
                    #val_fit = np.exp(fit_params[0][0])/cur_max_N
                    #ax.plot(np.array(xaxis),np.exp(fit_params[0][0]+2*np.log(np.array(xaxis))),color='aquamarine',zorder=-10,ls='-',lw=10,label=r'$f(\sigma)='+str(val_fit)+r'\sigma^2 N_{max}$')
                    # WITHOUT FIT:
                    ax.plot(np.array(xaxis),2*cur_max_N*np.exp(2*np.log(np.array(xaxis))),color='aquamarine',zorder=-10,ls='-',lw=10,label=r'$f(\sigma_\mathrm{noise})=2N\sigma_\mathrm{noise}^2 $')
                else:
                    # WITH FIT:
                    #ax.plot(np.array(xaxis),val_fit*cur_max_N*np.exp(2*np.log(np.array(xaxis))),color='aquamarine',zorder=-10,ls='-',lw=10)
                    # WITHOUT:
                    ax.plot(np.array(xaxis),2*cur_max_N*np.exp(2*np.log(np.array(xaxis))),color='aquamarine',zorder=-10,ls='-',lw=10)
    load_filename = 'pt_localization_excitation_L={0}_maxN={1}_system_majorana_mapping_disorder_everystep=True_new_nsteps_definition_corrected_noise_rad={2}_trotter={3}_{4}'.format(str(sys_size_arr),str(max_N_arr),str(rad),str(trot),model_name[:5])     
    #if match_guess:
    #    load_filename += '_coe='+str(guess_coe)
    
    # Change tick sizes
    ax.tick_params(axis = 'both', which = 'major', labelsize = FONTSIZE_TICKS)
    
    ax.legend(loc='lower right',fontsize=FONTSIZE_LEGEND)
    ax.figure.savefig(load_filename+'.pdf', bbox_inches='tight')

## NOT NEEDED! FIG.X: No disorder, $L=100$, $N_{max}=100$

In [ ]:
# Define variables
disorder_xx_var_arr = [0]
static_disorder_xx_var_arr = [0]
max_N = 150
N_start = 1
radi = 1
to_trotter_radius = False
N_opt = list(range(N_start,max_N+1,2))
L_sizes = [150]
disorder_count = 1 # There is no disorder so we take only one sample
model_name = 'no_disorder_old'
print(N_opt)
print(model_name)

In [ ]:
save_disorder_data(L_sizes,N_opt,disorder_xx_var_arr,static_disorder_xx_var_arr,cur_disorder_count=disorder_count,_model_name=model_name,radiu=radi,to_trotter= to_trotter_radius)

In [ ]:
# Plot results
# Load the data
max_N =149
ax=plot_zero_disorder_match([L_sizes[0]],max_N,model_name,add1overL=False,fit_region=[10,74])
#ax.figure.savefig('figures/final_overleaf_definitions_10_06_2022/FIG2.pdf')

## DECIDED TO NOT SHOW: FIG.3(c): Part 1, Varying static disorder, $L = 20,45,70,95$, $N_{max}= 10$. Fit of $0.8\sigma^2$ plotted above.

In [ ]:
# Define variables
disorder_xx_var_arr = [0]#[0.1]
static_disorder_xx_var_arr = sorted(np.append(np.logspace(-3,-0.6,15),0))#sorted(np.append(2e-3*np.array(range(10,200,10)),0))
max_N = 10#30
N_start = 1
radi = 1
to_trotter_radius = False
N_opt = [max_N]#list(range(N_start,max_N+1,5))
L_sizes = [20,45,70,95]
disorder_count = 500
is_disorder_every_step = False
model_name = 'static_disorder_with_everystep='+str(disorder_xx_var_arr[0])+'_for_scaling_only_end_step_disorder_count=500'
print(N_opt)
print(static_disorder_xx_var_arr)
print(model_name)

In [ ]:
save_disorder_data(L_sizes,N_opt,disorder_xx_var_arr,static_disorder_xx_var_arr,cur_disorder_count=disorder_count,_model_name=model_name,radiu=radi,to_trotter= to_trotter_radius)

In [ ]:
FONTSIZE_AXIS = 30
FONTSIZE_TICKS = 26

L_sizes_arr = [20,45,70,95]
max_N_arr = 10
#model_name = 'static_disorder_with_everystep=0_for_scaling_only_end_step_disorder_count=500'
# TODO: Enable this after testing
plot_sat_values_with_fixed_slope(L_sizes_arr,max_N_arr,match_guess=False)
# TEST NOW WHAT IS THE FITTED COEFFICIENT AND TRY TO ESTIMATE THE DEVIATION
#plot_sat_values_with_fixed_slope(L_sizes_arr,max_N_arr,match_guess=False)

## FIG.3(d): Part 2, Varying static disorder, $L=30$, $N_{max} = 20,100,200,400$. Fit of $0.8\sigma^2$ plotted above.

In [ ]:
# Define variables
disorder_xx_var_arr = [0]#[0.1]
static_disorder_xx_var_arr = sorted(np.append(np.logspace(-3,-0.6,20),0))#sorted(np.append(2e-3*np.array(range(10,200,10)),0))
max_N = 1000#30
N_start = 1
radi = 1
to_trotter_radius = False
N_opt = [20,100,200,400]
L_sizes = [30]
# Was 100 but updated the disorder count to 500 after the suggestion of Bram 21.07.22
disorder_count = 500#100
is_disorder_every_step = False
model_name = 'static_disorder_with_everystep='+str(disorder_xx_var_arr[0])+'_for_scaling_only_end_step_disorder_count='+str(disorder_count)
print(N_opt)
print(static_disorder_xx_var_arr)
print(model_name)

In [ ]:
save_disorder_data(L_sizes,N_opt,disorder_xx_var_arr,static_disorder_xx_var_arr,cur_disorder_count=disorder_count,_model_name=model_name,radiu=radi,to_trotter= to_trotter_radius)

In [ ]:
FONTSIZE_AXIS = 30
FONTSIZE_TICKS = 26

L_sizes_arr = [30]
max_N_arr = [20,100,200,400]
#model_name = 's_d_w_s='+str(disorder_xx_var_arr[0])+'_c=100'
plot_sat_values_with_fixed_slope(L_sizes_arr,max(max_N_arr),match_guess=False,apply_opacity_error=True)

## Fig.2: Varying each step disorder $\sigma$, varying $N_{steps}$, $L=70$ with theoretical fits

In [ ]:
# Define variables
# Update 31.05.2022: Remove the 0.3 and 0.5 as they don't really combine well with the ansatz scaling
disorder_xx_var_arr = [0,1e-2,3e-2,5e-2,1e-1]#,3e-1,5e-1]#,1]
static_disorder_xx_var_arr = [0]
max_N = 100#30
N_start = 1
N_opt = list(range(N_start,max_N+1,1))
model_name = 'everystep_disorder'
radi = 1
to_trotter_radius = False
L_sizes = [120]
disorder_count = 10
print(N_opt)
print(disorder_xx_var_arr)
print(model_name)

In [ ]:
save_disorder_data(L_sizes,N_opt,disorder_xx_var_arr,static_disorder_xx_var_arr,cur_disorder_count=disorder_count,_model_name=model_name,radiu=radi,to_trotter= to_trotter_radius)

In [ ]:
# Define xaxis and plot
fig, ax = plt.subplots(figsize=(10, 6))

   
counts_points = 100
N_start = 1
max_N = 100
L_size = 120
xaxis,sigmas,defects,defects_err=data_kz_scaling_disorder(L_size,max_N,model_name)
sigmas = [x[0] for x in sigmas]
print(xaxis)
#plt.plot(xaxis,0.35*xaxis**-0.5,'g--')
for s in range(len(sigmas)):
    if sigmas[s] == 0:
        fit_region=[50,99]
        if (len(fit_region)<2) : fit_region=range(0,len(xaxis))
        if (len(fit_region)==2) : fit_region=range(fit_region[0],fit_region[1]+1)
        #fit_params = scipy.optimize.curve_fit(lambda x,a,b: a+b*x, np.log(xaxis[fit_region]), np.log(np.array(defects[s])[fit_region]))
        #a = fit_params[0][0]
        #b = fit_params[0][1]
        #print((a,b))
        #ax.plot(xaxis,np.exp(a+b*np.log(xaxis)),c='r',ls='--',lw=5,label = r'$N^{{{0}}}$'.format(str(b)[0:5]))

        #fit_params = scipy.optimize.curve_fit(lambda x,a,b: a-0.5*x, np.log(xaxis[fit_region]), np.log(np.array(defects[s])[fit_region]))
        #a = fit_params[0][0]
        ##b = fit_params[0][1]
        #print(a)
        #ax.plot(xaxis,np.exp(a-0.5*np.log(xaxis)),c='r',ls='--',lw=5,label = r'${0}N^{{-0.5}}$'.format(str(np.exp(a))[0:5]))


        fit_params = scipy.optimize.curve_fit(lambda x,a: a*x**(-0.5), xaxis[fit_region], np.array(defects[s])[fit_region])
        a = fit_params[0][0]
        err = np.sqrt(fit_params[1][0][0])
        #percent_error = max(np.exp(err)-1,1-np.exp(-err))
        print("Prefactor:{0}".format(a))
        #print("Percent error:{0}".format(percent_error))
        print("{0}---{1}".format(a-err,a+err))
        # See also https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html:
        # To compute one standard deviation errors on the parameters use perr = np.sqrt(np.diag(pcov)).
        print('standard deviation={0}'.format(str(err)))
        #b = fit_params[0][1]
        #print(a)
        ax.plot(xaxis,a*np.exp(-0.5*np.log(xaxis)),c='dimgray',ls='--',lw=5,label = r'$\mathrm{KZ}$')#r'${0:.3f}N^{{-0.5}}$'.format(a))

        # Plot also the 0 line
        #plt.errorbar(times,np.array(defects[s]),ls='--',label=r'$\sigma='+str(sigmas[s])+r'$',marker='o',markersize=4,capsize=2,yerr=defects_err[s])
        plt.errorbar(xaxis,np.array(defects[s]),ls='--',label=r'$\sigma_\mathrm{noise}=0$',marker='o',markersize=4,capsize=2,yerr=defects_err[s])

        #plt.plot(times,np.array(defects[0])+2*times*sigmas[s]**2,'k--',zorder=10) 
    else:
        fit_region=range(10,100)
        plt.errorbar(xaxis,np.array(defects[s]),ls='--',label=r'$\sigma_\mathrm{noise}='+str(sigmas[s])+r'$',marker='o',markersize=4,capsize=2,yerr=defects_err[s])
        # TODO: Check if we fit hee corresponding to the fit region????
        # Fit only for the lowest sigma
        if sigmas[s] == 1e-2:
            fit_params = scipy.optimize.curve_fit(lambda x,a: a*x*sigmas[s]**2, xaxis[fit_region], np.array(defects[s]-defects[0])[fit_region],sigma=np.array(defects_err[s])[fit_region])
            a = fit_params[0][0]
            err = np.sqrt(fit_params[1][0][0])
            #percent_error = (a+err)/a-1
            print('~~~~~~~~~ START sigma={0}~~~~~~~~~~~~~'.format(str(sigmas[s])))
            print("Prefactor:{0}".format(a))
            #print("Percent error:{0}".format(percent_error))
            print("{0}---{1}".format(fit_params[0][0]-err,fit_params[0][0]+err))
            # See also https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html:
            # To compute one standard deviation errors on the parameters use perr = np.sqrt(np.diag(pcov)).
            print('standard deviation={0}'.format(str(np.sqrt(np.diag(fit_params[1])))))


        plt.plot(xaxis,np.array(defects[0])+a*xaxis*sigmas[s]**2,'k--',zorder=3) 
#Set log scale
plt.yscale('log')
plt.xscale('log') 

ax.set_yticks([0.02,0.1,0.5])
ax.set_yticklabels([str(0.02),str(0.1),str(0.5)])
plt.legend(loc="lower left",fontsize=FONTSIZE_LEGEND-1,ncol=2)
#plt.set_xticks(xaxis) # NO MINOR FLAG FOR NEWER MATPLOTLIB
ax.set_ylabel(r'$d$',fontsize=FONTSIZE_AXIS)
ax.set_xlabel(r'$N$',fontsize=FONTSIZE_AXIS)



# Get this figure without the x and y limits
#plt.xlim([4,100])
plt.ylim([0.02,0.6])
plt.xticks(fontsize=FONTSIZE_TICKS)
plt.yticks(fontsize=FONTSIZE_TICKS)
#plt.savefig('figures/final_overleaf_definitions_10_06_2022/FIG2.pdf')
plt.savefig('figures/final_overleaf_definitions_13_09_2022/FIG2.pdf')

plt.show()

## Fig.4: Optimal number of steps with theoretical fit

In [ ]:
# Define variables
# Update 31.05.2022: Remove the 0.3 and 0.5 as they don't really combine well with the ansatz scaling
disorder_xx_var_arr = sorted(np.append(np.logspace(-3,-0.3,20),0))#,3e-1,5e-1]#,1]
static_disorder_xx_var_arr = [0]
max_N = 2000#30
N_start = 1
N_opt = list(range(N_start,10,1))+list(range(10,100,2))+list(range(100,1000,40))+list(range(1000,max_N+1,200))
model_name = 'everystep_disorder_for_optimal'
radi = 1
to_trotter_radius = False
L_sizes = [20,30]#[50]
disorder_count = 10
print(N_opt)
print(disorder_xx_var_arr)
print(model_name)

In [ ]:
save_disorder_data(L_sizes,N_opt,disorder_xx_var_arr,static_disorder_xx_var_arr,cur_disorder_count=disorder_count,_model_name=model_name,radiu=radi,to_trotter= to_trotter_radius)

In [ ]:
L_sizes_arr = [] # UNUSED VAR?
max_N_arr = [] # UNUSED VAR?
# Import data
model_name = 'everystep_disorder_for_optimal'
N_start = 1
max_N = 2000#740#591
#times = np.linspace(N_start,max_N,counts_points) # 1...100
L_size_arr = [20,30,50]#80#150


# Get minimas and plot
fig, ax = plt.subplots(figsize=(10, 6))
color_arr = ["tab:red","tab:blue","tab:green"]
for i in range(len(L_size_arr)):
    L_size = L_size_arr[i]
    xaxis,sigmas,defects,defects_err=data_kz_scaling_disorder(L_size,max_N,model_name)

    sigmas = [x[0] for x in sigmas]
    minima=[]
    lower_error = []
    upper_error = []
    print(sigmas)
    for s in range(len(sigmas)):
        # Skip 0,1e-3 and the maximal sigma
        if s==0 or s==len(sigmas)-1 or s==len(sigmas)-2:
            print(sigmas[s])
            sigmas[s]=-1
            continue
        minimum=np.min(defects[s])
        # Here we generate error bars by adding 5% error to the minimum value
        min_err_limit = 1.05*minimum
        # We check the min/max xaxis and draw an error bar accordingly
        res_limits = xaxis[np.where(defects[s]<=min_err_limit)]
        res = xaxis[np.where(defects[s]==minimum)][0]
        #print((min(res_limits),max(res_limits)))
        
        low_err = abs(min(res_limits)-res)
        high_err=abs(max(res_limits)-res)
        if low_err==0 and high_err==0:
            print(sigmas[s])
            sigmas[s]=-1
            continue
            
        minima.append(res)
        lower_error.append(low_err)
        upper_error.append(high_err)
    sigmas = [s for s in sigmas if s!=-1]
    print(minima)
    # From https://stackoverflow.com/a/62525717/5774432
    asymmetric_error = np.array(list(zip(lower_error, upper_error))).T

    plt.errorbar(sigmas,minima,yerr=asymmetric_error,c=color_arr[i],fmt='o',ms=8,capsize=8,markeredgewidth=3, elinewidth=3,label=r'$L={0}$'.format(str(L_size)))
    # Generate Fit
    err_fix = np.array([max(asymmetric_error[0][i],asymmetric_error[1][i]) for i in range(len(asymmetric_error[0]))])
    #print(err_fix)
    fit_params = scipy.optimize.curve_fit(lambda x,a: a*x**(-4.0/3.0), np.array(sigmas)[3:-6], np.array(minima)[3:-6],sigma=err_fix[3:-6])
    a = fit_params[0][0]

    print("~~~~~~~~~~~~~ for L={0}~~~~~~~~~~~~~~".format(str(L_size)))
    err = np.sqrt(fit_params[1][0][0])
    print("Prefactor:{0}".format(a))
    print("{0}---{1}".format(a-err,a+err))
    # See also https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html:
    # To compute one standard deviation errors on the parameters use perr = np.sqrt(np.diag(pcov)).
    print('standard deviation={0}'.format(err))
    
    #if L_size==20:
    #    plt.plot(sigmas,a*np.exp((-4.0/3.0)*np.log(sigmas)),c=color_arr[i],lw=5,label=r'$ {0:.2f}\sigma_\mathrm{{noise}}^{{-\frac{{4}}{{3}}}}$'.format(a))
    #else:
    #    plt.plot(sigmas,a*np.exp((-4.0/3.0)*np.log(sigmas)),c=color_arr[i],lw=5,label=r'$ {0:.3f}\sigma_\mathrm{{noise}}^{{-\frac{{4}}{{3}}}}$'.format(a))
    if L_size == 50:
        plt.plot(sigmas,a*np.exp((-4.0/3.0)*np.log(sigmas)),c=color_arr[i],zorder=-10,lw=4,ls='--',label=r'$ {0:.3f}\sigma_\mathrm{{noise}}^{{-\frac{{4}}{{3}}}}$'.format(a))
#plt.plot(sigmas,0.8/np.array(sigmas)**(4/3),label='$ 0.8\sigma_\mathrm{noise}^{-4/3}$')
#plt.ylim([1.5,200])
#plt.xlim([0.001,0.01])
plt.xlabel(r'$\sigma_\mathrm{noise}$',fontsize=FONTSIZE_AXIS)
plt.ylabel(r'$N_{\rm opt}$',fontsize=FONTSIZE_AXIS)

plt.xscale('log')
plt.yscale('log')

plt.xticks(fontsize=FONTSIZE_TICKS)
plt.yticks(fontsize=FONTSIZE_TICKS)

#plt.xlim((10**(-3),max(sigmas)+0.05))

plt.legend(loc="lower left",fontsize=FONTSIZE_LEGEND,ncol=1)

#plt.savefig('figures/final_overleaf_definitions_10_06_2022/FIG4.pdf')
plt.savefig('figures/final_overleaf_definitions_13_09_2022/FIG4.pdf')

plt.show()

## NEW REFEREE Q: Fig. X: Optimal number of steps as a function of $L$ for $\sigma_{\rm noise}=1e^{-1}$ 

In [ ]:
# Define variables
# As Emanuele suggested we try \sigma_noise=\sigma_disorder=0.1 which is in the experimental regime
disorder_xx_var_arr = [0.1]
static_disorder_xx_var_arr = [0.1]
max_N = 15 # 120 is derived from Fig. 4 above that we see N_opt to be ~100 for \sigma_{noise}=0.1. We don't think that disorder changes it much.
N_start = 1
N_opt = list(range(N_start,max_N+1,1))#list(range(N_start,10,1))+list(range(10,100,2))
radi = 1
to_trotter_radius = False
L_sizes = [4,5,6]#list(range(5,100,5))
disorder_count = 1000
model_name = 'optimal_for_varying_sys_sizes_count={0}'.format(str(disorder_count))
print(N_opt)
print(L_sizes)
print(disorder_xx_var_arr)
print(model_name)

In [ ]:
save_disorder_data(L_sizes,N_opt,disorder_xx_var_arr,static_disorder_xx_var_arr,cur_disorder_count=disorder_count,_model_name=model_name,radiu=radi,to_trotter= to_trotter_radius)

In [ ]:
# Import data
model_name = 'optimal_for_varying_sys_sizes_count=1000'
max_N = 15#740#591
#times = np.linspace(N_start,max_N,counts_points) # 1...100
L_size_arr = [4,5,6]#range(10,100,5)

# Get minimas and plot
fig, ax = plt.subplots(figsize=(10, 6))
color_arr = ["tab:red","tab:blue","tab:green"]
minima=[]
lower_error = []
upper_error = []
for i in range(len(L_size_arr)):
    L_size = L_size_arr[i]
    xaxis,sigmas,defects,defects_err=data_kz_scaling_disorder(L_size,max_N,model_name)

    sigmas = [x[0] for x in sigmas]

    print(sigmas)

    for s in range(len(sigmas)):
        # Skip 0,1e-3 and the maximal sigma
        #if s==0 or s==len(sigmas)-1 or s==len(sigmas)-2:
        #    print(sigmas[s])
        #    sigmas[s]=-1
        #    continue
        minimum=np.min(defects[s])
        # Here we generate error bars by adding 5% error to the minimum value
        min_err_limit = 1.05*minimum
        # We check the min/max xaxis and draw an error bar accordingly
        res_limits = xaxis[np.where(defects[s]<=min_err_limit)]
        res = xaxis[np.where(defects[s]==minimum)][0]
        #print((min(res_limits),max(res_limits)))
        
        low_err = abs(min(res_limits)-res)
        high_err=abs(max(res_limits)-res)
        #if low_err==0 and high_err==0:
        #    print("ERROR! NO ERROR BARS!")
        #    print(sigmas[s])
        #    sigmas[s]=-1
        #    continue
            
        minima.append(res)
        lower_error.append(low_err)
        upper_error.append(high_err)
    sigmas = [s for s in sigmas if s!=-1]
    print(minima)
# From https://stackoverflow.com/a/62525717/5774432
asymmetric_error = np.array(list(zip(lower_error, upper_error))).T

# Generate Fit
syn_error = np.array([max(asymmetric_error[0][i],asymmetric_error[1][i]) for i in range(len(asymmetric_error[0]))])

plt.errorbar(L_size_arr,minima,yerr=asymmetric_error,fmt='o',ms=8,capsize=8,markeredgewidth=3, elinewidth=3,label=r'$\sigma_{\rm noise}=\sigma_{\rm disorder}=0.1$')



#plt.ylim([1.5,200])
#plt.xlim([0.001,0.01])
plt.xlabel(r'$L$',fontsize=FONTSIZE_AXIS)
plt.ylabel(r'$N_{\rm opt}$',fontsize=FONTSIZE_AXIS)

#plt.xscale('log')
#plt.yscale('log')

plt.xticks(fontsize=FONTSIZE_TICKS)
plt.yticks(fontsize=FONTSIZE_TICKS)

plt.legend(loc="lower right",fontsize=FONTSIZE_LEGEND,ncol=1)

#plt.savefig('figures/final_overleaf_definitions_10_06_2022/FIG4.pdf')
#plt.savefig('figures/final_overleaf_definitions_13_09_2022/REFQ.pdf')

plt.show()

## NEW REFEREE Q: Fig. X2: Number of defects d vs N_{steps} to match with experiment for disorder=noise=0.1

In [ ]:
# Define variables
disorder_xx_var_arr = [0.1]
static_disorder_xx_var_arr = [0.1]
max_N = 30
N_start = 1
radi = 1
to_trotter_radius = False
N_opt = list(range(N_start,max_N+1,1))
L_sizes = [4,5,6]
disorder_count = 5000 # There is no disorder so we take only one sample
model_name = 'noise=0.1_disorder=0.1_experimental_sys_sizes_count={0}'.format(disorder_count)
print(N_opt)
print(model_name)

In [ ]:
numsets=-1
save_disorder_data(L_sizes,N_opt,disorder_xx_var_arr,static_disorder_xx_var_arr,cur_disorder_count=disorder_count,_model_name=model_name,radiu=radi,to_trotter= to_trotter_radius,setNum=numsets)

In [ ]:
# Plot results
# Define xaxis and plot
fig, ax = plt.subplots(figsize=(10, 6))

N_start = 1
max_N = 30
L_sizes = [4,5,6]


#print(xaxis)
#plt.plot(xaxis,0.35*xaxis**-0.5,'g--')
for i in range(len(L_sizes)):
    L_size = L_sizes[i]
    # Load the data
    xaxis,sigmas,defects,defects_err=data_kz_scaling_disorder(L_size,max_N,model_name)
    print(xaxis)
    #print(defects)
    #fit_region=range(10,100)
    for s in range(len(sigmas)):
        print(defects_err[s]/np.sqrt(5000))
        print(sigmas[s])
        if sigmas[s][0] == 0 or sigmas[s][1]==0:
            continue
        plt.errorbar(xaxis,np.array(defects[s]),ls='--',label=r'$L={0}$'.format(str(L_size)),marker='o',markersize=4,capsize=6,yerr=defects_err[s]/np.sqrt(5000))

    # This is the fit
    #plt.plot(xaxis,np.array(defects[0])+a*xaxis*sigmas[s]**2,'k--',zorder=3) 
#Set log scale
#plt.yscale('log')
#plt.xscale('log') 

#ax.set_yticks([0.02,0.1,0.5])
#ax.set_yticklabels([str(0.02),str(0.1),str(0.5)])
plt.legend(loc="lower right",fontsize=FONTSIZE_LEGEND,ncol=1)
#plt.set_xticks(xaxis) # NO MINOR FLAG FOR NEWER MATPLOTLIB
ax.set_ylabel(r'$d$',fontsize=FONTSIZE_AXIS)
ax.set_xlabel(r'$N$',fontsize=FONTSIZE_AXIS)
ax.text(2,0.44,r'$\sigma_\mathrm{disorder}=\sigma_\mathrm{noise}=0.1$',fontsize=FONTSIZE_LEGEND+5)


# Get this figure without the x and y limits
#plt.xlim([4,100])
#plt.ylim([0.02,0.6])
plt.xticks(fontsize=FONTSIZE_TICKS)
plt.yticks(fontsize=FONTSIZE_TICKS)
#ax.figure.savefig('figures/final_overleaf_definitions_13_09_2022/REFQX2_count={0}.pdf'.format(disorder_count))

plt.show()

### Genrating data for this figure - SKIP THIS SECTION

In [ ]:
# New data format
def data_kz_scaling_disorder(sys_size,max_N,model_name,xaxis=None):
    # Load data from files
    load_filename = 'pt_localization_excitation_L={0}_maxN={1}_system_majorana_mapping_disorder_everystep=True_rad=1_trotter=False_{2}'.format(str(sys_size),str(max_N),model_name)
    yaxis_dict = np.load(load_filename+'_endstep_new_nsteps_definition_corrected_noise_data_mean_dict.npy', allow_pickle=True).item()
    yaxis_std_dict = np.load(load_filename+'_endstep_new_nsteps_definition_corrected_noise_data_std_dict.npy', allow_pickle=True).item()

    # Choose the xaxis to be the N's of the first in the dictionary
    if xaxis is None:
        xaxis = list(list(yaxis_dict.values())[0].keys())
    
    sigmas = []
    defects = []
    defects_err = []
    for k,yaxis in sorted(yaxis_dict.items()):
        sigmas.append(k)
        l_yaxis = []
        l_yerr_arr = []
        for n_opt in xaxis:
            l_yaxis.append(yaxis[n_opt])
            l_yerr_arr.append(yaxis_std_dict[k][n_opt])
        defects.append(np.array(l_yaxis))
        #defects.append(np.array(l_yaxis)*(sys_size-1)/(sys_size)+(1/(2*sys_size)))
        #defects.append(np.array(l_yaxis)+(1/(2*sys_size)))
        defects_err.append(np.array(l_yerr_arr))
    
    return np.array(xaxis),np.array(sigmas),np.array(defects),np.array(defects_err)

In [ ]:
count=5000
model_name = 'noise=0.1_disorder=0.1_experimental_sys_sizes_count='+str(count)
max_N = 30
L_sizes = [4,5,6]
for L_size in L_sizes:
    xaxis,sigmas,defects,defects_err=data_kz_scaling_disorder(L_size,max_N,model_name)
    saved_avg_arr = np.array([np.array(defects[0]),np.array(defects_err[0]/np.sqrt(count))])
    np.save(model_name+'_L={0}'.format(str(L_size)),saved_avg_arr)

## Fig.3(b): Part 1, Varying every step noise, $L = 30,45,60$, $N_{max}= 200$. Fit of $\sigma^2$ plotted above.

In [ ]:
# Define variables
disorder_xx_var_arr = sorted(np.append(1e-5*np.array(range(10,800,50)),0))
static_disorder_xx_var_arr = [0]
max_N = 3000#30
N_start = 1
radi = 1
to_trotter_radius = False
N_opt = [max_N]#list(range(N_start,max_N+1,5))
L_sizes = [20,30,40]
disorder_count = 5
is_disorder_every_step = False
model_name = 'noise='+str(disorder_xx_var_arr[0])+'_count='+str(disorder_count)
print(N_opt)
print(disorder_xx_var_arr)
print(model_name)

In [ ]:
save_disorder_data(L_sizes,N_opt,disorder_xx_var_arr,static_disorder_xx_var_arr,cur_disorder_count=disorder_count,_model_name=model_name,radiu=radi,to_trotter= to_trotter_radius)

In [ ]:
FONTSIZE_AXIS = 30
FONTSIZE_TICKS = 26

plot_sat_values_with_fixed_slope_noise(L_sizes,max_N,match_guess=False)

## Fig.3(a): Part 2, Varying noise, $L=20$, $N_{max} = 2000--4000$ for $\sigma = 0,5e-5,1e-4,5e-4$.

In [ ]:
# Define variables
disorder_xx_var_arr = [0,1e-4,5e-4,1e-3]
static_disorder_xx_var_arr = [0]
#max_N = 4000
#N_start = 2000
radi = 1
to_trotter_radius = False
N_opt = np.ceil(sorted(np.append(np.logspace(1,4,30),0))).astype(int)
L_sizes = [20]
disorder_count = 5
is_disorder_every_step = False
model_name = 'alt_vn_noise='+str(disorder_xx_var_arr[0])
print(N_opt)
print(disorder_xx_var_arr)
print(model_name)

In [ ]:
save_disorder_data(L_sizes,N_opt,disorder_xx_var_arr,static_disorder_xx_var_arr,cur_disorder_count=disorder_count,_model_name=model_name,radiu=radi,to_trotter= to_trotter_radius)

In [ ]:
FONTSIZE_AXIS = 30
FONTSIZE_TICKS = 26

L_sizes = [20]
max_N = 10000
model_name = 'alt_vn_noise=0'
plot_sat_values_with_fixed_slope_noise_nmax(L_sizes[0],max_N,match_guess=False,cap_nmax=80)

##  NOT NEEDED, SKIP! Fig.3(b): Part 2 - Alternative, Varying noise, $L=20$, $N_{max} = 2000,2500,3000,3500,4000$. Fit of $\sigma^2$ plotted above.

In [ ]:
# Define variables
disorder_xx_var_arr = sorted(np.append(1e-5*np.array(range(10,800,50)),0))
static_disorder_xx_var_arr = [0]
max_N = 4000
N_start = 2000
radi = 1
to_trotter_radius = False
N_opt = [200,800,1400,2800,4000]
L_sizes = [20]
disorder_count = 5
is_disorder_every_step = False
model_name = 'varying_noise_alternative'
print(N_opt)
print(disorder_xx_var_arr)
print(model_name)

In [ ]:
save_disorder_data(L_sizes,N_opt,disorder_xx_var_arr,static_disorder_xx_var_arr,cur_disorder_count=disorder_count,_model_name=model_name,radiu=radi,to_trotter= to_trotter_radius)

In [ ]:
L_sizes = [20]
max_N = 4000
model_name = 'varying_noise_alternative'
plot_sat_values_with_fixed_slope_noise_alternative(L_sizes,max_N,match_guess=False)

## Fig. 7: Plotting the density of exitations versus the number of steps for many system sizes $L=4, 8, 16, 32, 64, 128$

In [ ]:
# Define variables
disorder_xx_var_arr = [0]
static_disorder_xx_var_arr = [0]
max_N = 120
N_start = 1
radi = 1
to_trotter_radius = False
N_opt = list(range(N_start,max_N+1,1))
L_sizes = [4,8,16,32,64,128,256]
disorder_count = 1 # There is no disorder so we take only one sample
model_name = 'no_disorder_many_different_system_sizes'
print(N_opt)
print(model_name)

In [ ]:
save_disorder_data(L_sizes,N_opt,disorder_xx_var_arr,static_disorder_xx_var_arr,cur_disorder_count=disorder_count,_model_name=model_name,radiu=radi,to_trotter= to_trotter_radius)

In [ ]:
# Plot results
# Load the data
max_N =120
ax=plot_zero_disorder_match(L_sizes,max_N,model_name,add1overL=False,fit_region=[50,119])
#ax.figure.savefig('figures/final_overleaf_definitions_10_06_2022/FIG7.pdf')
ax.figure.savefig('figures/final_overleaf_definitions_13_09_2022/FIG7.pdf')

# Qiskit implementation for adiabatic sweeping

In [ ]:
# NOW IT USES NEW N_STEPS DEFINITION HERE

from qiskit.tools.visualization import plot_histogram
from functools import reduce
# Import API for cloud computation
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, execute, IBMQ, transpile
# Enable when BasicAer not found
#from qiskit import Aer
from qiskit.quantum_info import Pauli
from qiskit.providers.aer import AerSimulator
from qiskit.providers.ibmq import least_busy
from qiskit.tools.monitor import job_monitor, backend_monitor, backend_overview
from qiskit.providers.aer import noise
from qiskit import QuantumRegister, QuantumCircuit, Aer
from qiskit.quantum_info import Statevector
import os.path
from datetime import date
from qiskit import Aer, transpile
import math

In [ ]:
# Font definitions
FONTSIZE_LEGEND = 22
FONTSIZE_AXIS = 24
FONTSIZE_TITLE = 24
FONTSIZE_TICKS = 20

### INITIALIZATION ###

IS_PBC = True

# Number of qubits for the simulation
QUBITS_NUM = 16
SHOTS = 30000 # We test with 8000 shots and then we will increase it to 30000
SIM_NUM = 5
# BATCH_SIZE = 5 for the real quantum computer as there is a queue
BATCH_SIZE = SIM_NUM

FILE_OUT = False
# local = simulation without noise
RUN_TYPE = 'local' # 'local', 'realqc', 'noise'

TODAY = str(date.today())
FILENAME = 'counts_z2_1system_dist_'+RUN_TYPE+'_'+TODAY+'_'+str(QUBITS_NUM)

In [ ]:
# Loading cerdentials
IBMQ.load_account()

In [ ]:
def calc_ener_from_bit_string(cur_bits):
    # cur_bits is an array of integers representing bits
    cur_ener = 0
    # The energy is with OBC, the minimum energy is -L+1
    for i in range(len(cur_bits)-1):
        # We add 1 for each domain wall and -1 if there is no domain wall like the Hamiltonian
        cur_ener += -1*(1-2*cur_bits[i])*(1-2*cur_bits[i+1])
    if IS_PBC:
        cur_ener += -1*(1-2*cur_bits[len(cur_bits)-1])*(1-2*cur_bits[0])
    return cur_ener
def calc_dw_from_bit_string(cur_bits):
    # cur_bits is an array of integers representing bits
    cur_dw = 0
    # The energy is with OBC, the minimum energy is -L+1
    for i in range(len(cur_bits)-1):
        # We add 1 for each domain wall and -1 if there is no domain wall like the Hamiltonian
        cur_dw += 1-(1-2*cur_bits[i])*(1-2*cur_bits[i+1])
    if IS_PBC:
        cur_dw += 1-(1-2*cur_bits[len(cur_bits)-1])*(1-2*cur_bits[0])
    return 0.5*cur_dw
def calc_dw_cors_from_bit_string(cur_bits):
    # Returns array of <N_0N_i>
    # cur_bits is an array of integers representing bits
    
    dw_at_start = (1-(1-2*cur_bits[0])*(1-2*cur_bits[1]))
    # The energy is with OBC, the minimum energy is -L+1
    max_dws = len(cur_bits)-1
    if IS_PBC:
        max_dws = len(cur_bits)
    cur_cors = np.zeros(max_dws)
    for i in range(max_dws):
        # We add the multiplication of domain walls which is 1 if both exist and -1 if one is not and the other does
        cur_cors[i] = dw_at_start*(1-(1-2*cur_bits[i])*(1-2*cur_bits[(i+1)%len(cur_bits)]))
    return 0.25*np.array(cur_cors)
def calc_dw_avgs_from_bit_string(cur_bits):
    # Returns array of <N_0N_i>
    # cur_bits is an array of integers representing bits
    # The energy is with OBC, the minimum energy is -L+1
    max_dws = len(cur_bits)-1
    if IS_PBC:
        max_dws = len(cur_bits)
    cur_cors = np.zeros(max_dws)
    for i in range(max_dws):
        # We add the multiplication of domain walls which is 1 if both exist and -1 if one is not and the other does
        cur_cors[i] = 0.5*(1-(1-2*cur_bits[i])*(1-2*cur_bits[(i+1)%len(cur_bits)]))
    return np.array(cur_cors)
def calc_zzzz_cors_from_bit_string(cur_bits):
    # Returns array of Z_1Z_2Z_jZ_{j+1}
    # cur_bits is an array of integers representing bits
    
    dw_at_start = (1-2*cur_bits[0])*(1-2*cur_bits[1])
    # The energy is with OBC, the minimum energy is -L+1
    max_dws = len(cur_bits)-1
    if IS_PBC:
        max_dws = len(cur_bits)
    cur_cors = np.zeros(max_dws)
    for i in range(max_dws):
        # We add the multiplication of domain walls which is 1 if both exist and -1 if one is not and the other does
        cur_cors[i] = dw_at_start*((1-2*cur_bits[i])*(1-2*cur_bits[(i+1)%len(cur_bits)]))
    return np.array(cur_cors)

In [ ]:
def zz_int(circ,q,i,j,alpha):
    # Choose i->j such that it is in the connectivity direction
    circ.cx(q[i],q[j])
    circ.rz(2*alpha,q[j])
    circ.cx(q[i],q[j])

# from helper.py
def measure_partial_system(circ, qqubit, cbit):
    for i, qi in enumerate(qqubit):
        # measure qubit
        circ.measure(qi, cbit[i])

def prepare_plus_state(circ, q):
    """
    From |0> state to |+> state.

    Args:
        circ : the quantum circuit
        q : the register
    """
    for qi in q:
        circ.h(qi)

def measure_dw_density(counts):
    # NOT USED
    '''
    Return the expected number of domain walls N_ex = 0.5\sum_i 1-<Z_iZ_{i+1}>
    '''

    sys_size = len(list(list(counts.keys())[0]))

    # Array for <Z_iZ_{i+1}>
    zz_arr = np.zeros(sys_size-1)
    
    total_shots = 0
    for res in counts:
        # We reverse for convention: The FIRST index in res[0] is the result of the LAST qubit
        qubits_val = list(reversed(res))
        qubits_val = [int(qubits_val[i]) for i in range(len(qubits_val))]

        total_shots += counts[res]
        for i in range(len(qubits_val)-1):
            # Adding +1 if the spins are alligned and add -1 if the spins are anti alligned
            zz_arr[i] += (1-2*qubits_val[i])*(1-2*qubits_val[i+1])*counts[res]

    zz_arr = zz_arr*(1/total_shots)
    zz_arr = 0.5*(1-zz_arr)
    # Print the excitations
    # NOTE: We divide here by L-1
    return sum(zz_arr)/(sys_size-1)

def measure_dw_density_from_probs(probs):
    '''
    Return the expected number of domain walls density N_ex = 0.5\sum_i 1-<Z_iZ_{i+1}>/(L-1)
    '''
    sys_size = round(math.log2(len(probs)))
    bin_form = '0'+str(sys_size)+'b'
    exp_dw = 0
    for i in range(len(probs)):
        cur_l = str(format(i, bin_form))
        qubits_val = [int(cur_l[i]) for i in range(len(cur_l))]
        exp_dw += calc_dw_from_bit_string(qubits_val)*probs[i]
    # NOTE: We divide here by L-1
    if not IS_PBC:
        return exp_dw/(sys_size-1)
    else:
        return exp_dw/sys_size

def measure_dw_ener_dist(counts):
    '''
    Return the dist of domain walls domain walls N_ex = 0.5\sum_i 1-<Z_iZ_{i+1}>
    '''

    sys_size = len(list(list(counts.keys())[0]))

    # Array for <Z_iZ_{i+1}>
    zz_arr = np.zeros(sys_size-1)

    ret_dict = {}
    
    total_shots = sum(counts.values())
    for res in counts:
        # We reverse for convention: The FIRST index in res[0] is the result of the LAST qubit
        qubits_val = list(reversed(res))
        qubits_val = [int(qubits_val[i]) for i in range(len(qubits_val))]
        cur_ener = calc_ener_from_bit_string(qubits_val)
        cur_prob = counts[res]/total_shots
        ret_dict[res] = (cur_prob,cur_ener)
    return ret_dict
def measure_dw_dist_from_probs(probs):
    '''
    Return the dist of domain walls domain walls N_ex = 0.5\sum_i 1-<Z_iZ_{i+1}>
    '''
    ret_dict = {}
    bin_form = '0'+str(round(math.log2(len(probs))))+'b'
    for i in range(len(probs)):
        cur_l = str(format(i, bin_form))
        qubits_val = [int(cur_l[i]) for i in range(len(cur_l))]
        cur_prob = probs[i]
        cur_ener = calc_ener_from_bit_string(qubits_val)
        ret_dict[cur_l] = (cur_prob,cur_ener)
        #print(ret_dict[cur_l])
    return ret_dict
        
    sys_size = len(list(list(counts.keys())[0]))

    # Array for <Z_iZ_{i+1}>
    zz_arr = np.zeros(sys_size-1)

    ret_dict = {}
    
    total_shots = sum(counts.values())
    for res in counts:
        # We reverse for convention: The FIRST index in res[0] is the result of the LAST qubit
        qubits_val = list(reversed(res))
        qubits_val = [int(qubits_val[i]) for i in range(len(qubits_val))]
        cur_ener = calc_ener_from_bit_string(qubits_val)
        cur_prob = counts[res]/total_shots
        ret_dict[res] = (cur_prob,cur_ener)
    return ret_dict
def measure_dw_int_dist_from_probs(probs,funtype=calc_dw_cors_from_bit_string):
    '''
    Return the dist of domain walls domain walls N_ex[i] = <Z_iZ_{i+1}Z_iZ_{i+1}>
    '''
    sys_size = round(math.log2(len(probs)))
    bin_form = '0'+str(sys_size)+'b'
    exp_dw_int = np.zeros(sys_size)
    for i in range(len(probs)):
        cur_l = str(format(i, bin_form))
        qubits_val = [int(cur_l[i]) for i in range(len(cur_l))]
        exp_dw_int += funtype(qubits_val)*probs[i]
    # NOTE: We divide here by L-1 for OBC
    #if not IS_PBC:
    #    return exp_dw/(sys_size-1)
    #else:
    #    return exp_dw/sys_size
    return exp_dw_int

In [ ]:
def apply_floquet(circ,q,alpha,beta):
    # Apply X rotations
    # Creates exp(-i alpha X_i)

    # Let us first apply ZZ and then X

    # Apply ZZ
    for i in range(len(q)-1):
        # Creates exp(-i beta z_i z_{i+1})
        zz_int(circ,q,i,i+1,beta)
    if IS_PBC:
        zz_int(circ,q,len(q)-1,0,beta)
    # Apply X

    for i in range(len(q)):
        # Creates exp(-i alpha X_i)
        circ.rx(2*alpha,q[i])
def apply_floquet_arr(circ,q,alpha,beta):
    # Apply X rotations
    # Creates exp(-i alpha X_i)

    # Let us first apply ZZ and then X

    # Apply ZZ
    for i in range(len(q)-1):
        # Creates exp(-i beta z_i z_{i+1})
        zz_int(circ,q,i,i+1,beta[i])
    if IS_PBC:
        zz_int(circ,q,len(q)-1,0,beta[len(q)-1])
    # Apply X

    for i in range(len(q)):
        # Creates exp(-i alpha X_i)
        circ.rx(2*alpha[i],q[i])

In [ ]:
def run_circ_measure_prob_dist(circ,simulation_num,batch_size, run_type, file_name):
    #dws = [None for i in range(simulation_num)]
    #jobs = [[None] for i in range(simulation_num)]
    #counts = [[None] for i in range(simulation_num)]

    # Running the circuit on quantum computer/simulator

    #if RUN_TYPE == 'noise':
    #    # Choose a real device to get the noise from
    #    device = IBMQ.get_provider().get_backend('ibmq_16_melbourne')
    #    coupling_map = device.configuration().coupling_map
    #    # TODO: define properties
    #   noise_model = noise.device.basic_device_noise_model(properties) # some random errors about the prob. here
    #   basis_gates = noise_model.basis_gates
    #    #print(basis_gates)
    #    #exit()
    #    # print(noise_model.as_dict())

    #if RUN_TYPE != 'realqc':
    #    backend = Aer.get_backend('qasm_simulator')
    #else:
    #    backend = IBMQ.get_backend('ibmq_16_melbourne')
    #    max_credits = 5        # Maximum number of credits to spend on executions.
    #    backend_monitor(backend)
    backend = Aer.get_backend('aer_simulator_statevector')
    outputstate = backend.run(circ).result().get_statevector()
    probs = Statevector(outputstate).probabilities()
    dw_int = measure_dw_int_dist_from_probs(probs,funtype=calc_dw_cors_from_bit_string)
    dw_avg = measure_dw_int_dist_from_probs(probs,funtype=calc_dw_avgs_from_bit_string)
    dw_int_minus_avgs = [dw_int[i] - dw_avg[0]*dw_avg[i] for i in range(len(dw_int))]
    return measure_dw_dist_from_probs(probs),measure_dw_density_from_probs(probs),dw_int,dw_int_minus_avgs
    #completed_exps = 0
    #while(completed_exps<simulation_num):
    #    # Run all the jobs in parallel
    #    for i in range(batch_size):
    #        jobs[completed_exps+i] = execute(outcirc, backend=backend, shots=SHOTS)
    #        #if run_type == 'noise':
    #        #    jobs[completed_exps+i] = execute(outcirc, backend=backend, shots=SHOTS, noise_model=noise_model,coupling_map=coupling_map, basis_gates=basis_gates)
    #        #elif run_type == 'local':
    #        #    jobs[completed_exps+i] = execute(outcirc, backend=backend, shots=SHOTS) # optimization_level=2
    #        #else:
    #        #    jobs[completed_exps+i] = execute(outcirc, backend=backend, shots=SHOTS, max_credits=max_credits)
    #
    #   for i in range(batch_size):
    #       job_monitor(jobs[completed_exps+i])
    #        sim_result = jobs[completed_exps+i].result()
    #        counts[completed_exps+i] = sim_result.get_counts(outcirc)
    #        if FILE_OUT:
    #            with open(file_name+'.txt','a+') as f:
    #                f.write(str(counts[completed_exps+i]))
    #                f.write('\n')
    #        # Measure ENERGY DENSITY DIST.
    #        dws[completed_exps+i] = measure_dw_ener_dist(counts[completed_exps+i])
    #    completed_exps += batch_size
    #return dws

def run_circ_measure_dw_probs(circ):
    backend = Aer.get_backend('aer_simulator_statevector')
    outputstate = backend.run(circ).result().get_statevector()
    probs = Statevector(outputstate).probabilities()
    return measure_dw_density_from_probs(probs)

def run_circ_measure_dw(circ,simulation_num,batch_size, run_type, file_name):
    dws = [None for i in range(simulation_num)]
    jobs = [[None] for i in range(simulation_num)]
    counts = [[None] for i in range(simulation_num)]

    # Running the circuit on quantum computer/simulator

    if RUN_TYPE == 'noise':
        # Choose a real device to get the noise from
        device = IBMQ.get_provider().get_backend('ibmq_16_melbourne')
        coupling_map = device.configuration().coupling_map
        # TODO: define properties
        noise_model = noise.device.basic_device_noise_model(properties) # some random errors about the prob. here
        basis_gates = noise_model.basis_gates
        #print(basis_gates)
        #exit()
        # print(noise_model.as_dict())

    if RUN_TYPE != 'realqc':
        backend = Aer.get_backend('qasm_simulator')
    else:
        backend = IBMQ.get_backend('ibmq_16_melbourne')
        max_credits = 5        # Maximum number of credits to spend on executions.
        backend_monitor(backend)
    completed_exps = 0
    while(completed_exps<simulation_num):
        # Run all the jobs in parallel
        for i in range(batch_size):
            if run_type == 'noise':
                jobs[completed_exps+i] = execute(outcirc, backend=backend, shots=SHOTS, noise_model=noise_model,coupling_map=coupling_map, basis_gates=basis_gates)
            elif run_type == 'local':
                jobs[completed_exps+i] = execute(outcirc, backend=backend, shots=SHOTS) # optimization_level=2
            else:
                jobs[completed_exps+i] = execute(outcirc, backend=backend, shots=SHOTS, max_credits=max_credits)

        for i in range(batch_size):
            job_monitor(jobs[completed_exps+i])
            sim_result = jobs[completed_exps+i].result()
            counts[completed_exps+i] = sim_result.get_counts(outcirc)
            if FILE_OUT:
                with open(file_name+'.txt','a+') as f:
                    f.write(str(counts[completed_exps+i]))
                    f.write('\n')
            # Measure DW
            dws[completed_exps+i] = measure_dw_density(counts[completed_exps+i])

        completed_exps += batch_size
    mean = np.mean(dws)
    stddev = np.std(dws)
    print("Mean:" + str(mean))
    print("Deviation:" + str(stddev))
    return mean, stddev

In [ ]:
from enum import Enum

class GetBetaOption(Enum):
    lin_dist_fit = 1
    thermal_fit = 2

def get_beta(cur_dws,fit_op = GetBetaOption.lin_dist_fit):
    beta_arr = []
    std_arr = []
    for cur_sim in cur_dws:
        cur_sim = cur_sim.values()
        cur_x = [x[1] for x in cur_sim] # This is the energy
        cur_y = [x[0] for x in cur_sim] # This is the probability

        if fit_op == GetBetaOption.thermal_fit:
            # Prepare array of (E_x,average probability) to fit with Boltmann distribution
            blotz_fit_dist = dict()
            for cur_res in cur_sim:
                cur_e = cur_res[1]
                cur_p = cur_res[0]
                if not cur_e in blotz_fit_dist:
                    blotz_fit_dist[cur_e] = []
                blotz_fit_dist[cur_e].append(cur_p)
            
            dos_num_arr = []
            # Averaging the probabilities
            for cur_res in blotz_fit_dist:
                dos_num_arr.append(len(blotz_fit_dist[cur_res]))
                blotz_fit_dist[cur_res] = np.sum(blotz_fit_dist[cur_res])

            boltz_x = np.array(sorted(list(blotz_fit_dist.keys())))
            boltz_y = np.array([blotz_fit_dist[x] for x in boltz_x])

            # Get the boltzmann temprature
            fit_region = range(int(len(boltz_x)/2))
            #fit_params = scipy.optimize.curve_fit(lambda x,beta,c: c-1*beta*x, boltz_x[fit_region], np.log(boltz_y[fit_region]))
            #beta = fit_params[0][0]
            # TODSO: Fix this
            fit_params = scipy.optimize.curve_fit(lambda x,beta,c: c-1*beta*x, boltz_x[fit_region], np.log([boltz_y[i]/dos_num_arr[i] for i in range(len(boltz_y))])[fit_region])
            beta = fit_params[0][0]
            c = fit_params[0][1]
            std = np.sqrt(np.diag(fit_params[1]))[0]
            beta_arr.append(beta)
            std_arr.append(std)
        elif fit_op == GetBetaOption.lin_dist_fit:
            # Prepare array of (E_x,average probability) to fit with Boltmann distribution
            boltz_x = []
            boltz_y = []
            for cur_res in cur_sim:
                cur_x = cur_res[1]
                cur_y = cur_res[0]
                if cur_x < 0:
                    boltz_x.append(cur_x)
                    boltz_y.append(cur_y)
            boltz_x = np.array(boltz_x)
            boltz_y = np.array(boltz_y)

            p, cov = np.polyfit(boltz_x, np.log(boltz_y), 1,cov=True)  
            beta = -1*p[0]
            c= p[1]
            
            std = np.sqrt(np.diag(cov))[0] # standard deviation in beta
            beta_arr.append(beta)
            std_arr.append(std)
    return np.average(np.array(beta_arr)),np.max(np.array(std_arr))

In [ ]:
# PLOTS
class DWType(Enum):
    zz_cors = 1
    dw_int = 2
    dw_int_minus_avg = 3
# Plot the saturation values as a function of the disorder for static disorder
def plot_ener_dist_space(cur_dws,fit_op=GetBetaOption.thermal_fit,noise_model=None):
    # Initialize the figure and axes
    fig, ax = plt.subplots(figsize=(10, 10))
    #ax.set_ylabel(r'Excitations $N_{\rm ex}$',fontsize=FONTSIZE_AXIS)
    ax.set_xlabel(r'$E$',fontsize=FONTSIZE_AXIS)
    ax.set_ylabel(r'$|\langle x | \psi \rangle |^2$',fontsize=FONTSIZE_AXIS)
    
    # Set log scale
    ax.set_yscale('log')

    plt.tight_layout()

    for cur_sim in cur_dws:
        cur_sim = cur_sim.values()
        cur_x = [x[1] for x in cur_sim] # This is the energy
        cur_y = [x[0] for x in cur_sim] # This is the probability

        if fit_op == GetBetaOption.thermal_fit:

            # Prepare array of (E_x,average probability) to fit with Boltmann distribution
            blotz_fit_dist = dict()
            for cur_res in cur_sim:
                cur_e = cur_res[1]
                cur_p = cur_res[0]
                if not cur_e in blotz_fit_dist:
                    blotz_fit_dist[cur_e] = []
                blotz_fit_dist[cur_e].append(cur_p)
            
            # Averaging the probabilities
            dos_num_arr = []
            avg_prob = []
            for cur_res in blotz_fit_dist:
                dos_num_arr.append(len(blotz_fit_dist[cur_res]))
                #print(blotz_fit_dist[cur_res])
                avg_prob.append(np.average(blotz_fit_dist[cur_res]))
                blotz_fit_dist[cur_res] = np.sum(blotz_fit_dist[cur_res])
                
            boltz_x = np.array(sorted(list(blotz_fit_dist.keys())))
            dos_num_theory = [2*math.comb(-1*min(boltz_x),i)  for i in range(len(boltz_x))]

            print(dos_num_arr)
            print(dos_num_theory)
            print(boltz_x)
            boltz_y = np.array([blotz_fit_dist[x] for x in boltz_x])

            print(np.sum(boltz_y))
            #ax.scatter(cur_x,cur_y)
            ax.plot(boltz_x,boltz_y,ls='-',lw=4)

            # Get the boltzmann temprature
            fit_region = range(int(1*len(boltz_x)/2))
            # -1*min(boltz_x)+1 is the system size L
            #print(-1*min(boltz_x)+1) = L
            #print(len(boltz_y)) = L
            #print(np.log([boltz_y[i]/math.comb(-1*min(boltz_x)+1,i+1) for i in range(len(boltz_y))]))
            fit_params = scipy.optimize.curve_fit(lambda x,beta,c: c-1*beta*x, boltz_x[fit_region], np.log([boltz_y[i]/dos_num_arr[i] for i in range(len(boltz_y))])[fit_region])
            beta = fit_params[0][0]
            c = fit_params[0][1]
            #percent_error = (a+err)/a-1
            print("Prefactor:{0}".format(beta))
            # See also https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html:
            # To compute one standard deviation errors on the parameters use perr = np.sqrt(np.diag(pcov)).
            print('standard deviation={0}'.format(str(np.sqrt(np.diag(fit_params[1])))))
            fit_yaxis = [dos_num_arr[i]*np.exp(c-1*beta*boltz_x[i]) for i in range(len(boltz_x))]
            ax.plot(boltz_x,fit_yaxis,ls="--",lw=4,label=r"$\beta={0}$".format(str(beta)))
            ax.plot(boltz_x,np.array(avg_prob),lw=4)
            print(fit_yaxis[0])
            print(boltz_y[0])

            load_filename = FILENAME+'_plot_ener_dist_stepstotal='+str(N_steps)+'_noise_model='+str(noise_model)+'_thermal_fit'

        elif fit_op == GetBetaOption.lin_dist_fit:
            # Prepare array of (E_x,average probability) to fit with Boltmann distribution
            boltz_x = []
            boltz_y = []
            for cur_res in cur_sim:
                cur_x = cur_res[1]
                cur_y = cur_res[0]
                if cur_x < 0:
                    boltz_x.append(cur_x)
                    boltz_y.append(cur_y)
            boltz_x = np.array(boltz_x)
            boltz_y = np.array(boltz_y)
            
            p, cov = np.polyfit(boltz_x, np.log(boltz_y), 1,cov=True)  
            beta = -1*p[0]
            c= p[1]
            
            std = np.sqrt(np.diag(cov))[0] # standard deviation in beta

            fit_xaxis = np.unique(np.array(boltz_x))
            fit_yaxis = [c-beta*fit_xaxis[i] for i in range(len(fit_xaxis))]
            ax.scatter(boltz_x,boltz_y)
            print(fit_xaxis)
            print(fit_yaxis)
            ax.plot(np.array(fit_xaxis),np.exp(np.array(fit_yaxis)),lw=4,c="red")

            load_filename = FILENAME+'_plot_ener_dist_stepstotal='+str(N_steps)+'_dist_fit'
    #if match_guess:
    #    load_filename += '_coe='+str(guess_coe)
    if IS_PBC:
        load_filename += "_pbc"
    # Change tick sizes
    ax.tick_params(axis = 'both', which = 'major', labelsize = FONTSIZE_TICKS)
    
    ax.legend(loc='lower right',fontsize=FONTSIZE_LEGEND)
    ax.figure.savefig(load_filename+'.pdf', bbox_inches='tight')
# Plot the domain wall interaction
def plot_dw_int(cur_dw_int_dict,steps_arr=None,plot_type=DWType.zz_cors):
    fig, ax = plt.subplots(figsize=(10, 10))
    #ax.set_ylabel(r'Excitations $N_{\rm ex}$',fontsize=FONTSIZE_AXIS)
    ax.set_xlabel(r'$r$',fontsize=FONTSIZE_AXIS)
    if plot_type == DWType.zz_cors:
        ax.set_ylabel(r'$\langle Z_0Z_1Z_{r}Z_{r+1} \rangle $',fontsize=FONTSIZE_AXIS)
    elif plot_type == DWType.dw_int:
        ax.set_ylabel(r'$\langle N_0 N_{r} \rangle $',fontsize=FONTSIZE_AXIS)
    elif plot_type == DWType.dw_int_minus_avg:
        ax.set_ylabel(r'$\langle N_0 N_{r} \rangle - \langle N_0 \rangle \langle N_{r} \rangle$',fontsize=FONTSIZE_AXIS)

    # Set log scale
    #ax.set_yscale('log')

    steps_keys = list(cur_dw_int_dict.keys())
    steps_keys_len = len(steps_keys)
    sys_size = len(cur_dw_int_dict[steps_keys[0]])

    xaxis = np.array(list(range(sys_size)))[1:]

    if steps_arr is None:
        steps_arr = [steps_keys[i] for i in range(0,steps_keys_len,int(steps_keys_len/10))]
    for s in steps_arr:
        yaxis = np.array(cur_dw_int_dict[s])[1:]
        ax.plot(xaxis,yaxis,lw=4,marker='o',label=r'$N_{{\rm steps}}={0}$'.format(str(s)))

    load_filename = FILENAME+'_plot_dw_int_stepstotal='+str(max(list(cur_dw_int_dict.keys())))+'_steps_arr={0}_type={1}'.format(str(steps_arr),str(plot_type.name))
    plt.tight_layout()
    if IS_PBC:
        load_filename += "_pbc"
    # Change tick sizes
    ax.tick_params(axis = 'both', which = 'major', labelsize = FONTSIZE_TICKS)
    
    ax.legend(loc='lower right',fontsize=FONTSIZE_LEGEND)
    ax.figure.savefig(load_filename+'.pdf', bbox_inches='tight')
def plot_cor_mid(cur_dw_int_dict,steps_arr=None,plot_type=DWType.dw_int_minus_avg):
    fig, ax = plt.subplots(figsize=(10, 10))
    #ax.set_ylabel(r'Excitations $N_{\rm ex}$',fontsize=FONTSIZE_AXIS)
    ax.set_xlabel(r'$N_{\mathrm{steps}}$',fontsize=FONTSIZE_AXIS)
    #if plot_type == DWType.zz_cors:
    #    ax.set_ylabel(r'$\langle Z_0Z_1Z_{r}Z_{r+1} \rangle $',fontsize=FONTSIZE_AXIS)
    #elif plot_type == DWType.dw_int:
    #    ax.set_ylabel(r'$\langle N_0 N_{r} \rangle $',fontsize=FONTSIZE_AXIS)
    if plot_type == DWType.dw_int_minus_avg:
        ax.set_ylabel(r'$\langle N_0 N_{L/2} \rangle - \langle N_0 \rangle \langle N_{L/2} \rangle$',fontsize=FONTSIZE_AXIS)
    else:
        raise Exception("TO DO")
    # Set log scale
    #ax.set_xscale('log')
    ax.set_yscale('log')

    steps_keys = list(cur_dw_int_dict.keys())
    steps_keys_len = len(steps_keys)
    sys_size = len(cur_dw_int_dict[steps_keys[0]])


    xaxis = np.array(steps_keys[:])
    yaxis = np.array([np.array(cur_dw_int_dict[s])[int(math.floor(sys_size/2))] for s in xaxis])
    ax.plot(xaxis,yaxis,lw=4,marker='o')

    # Add simple linear fit
    fit_len = 10
    fit_params = scipy.optimize.curve_fit(lambda x,a,b: a+b*x, xaxis[:fit_len], np.log(yaxis)[:fit_len])
    fit_slope = fit_params[0][1]
    fit_shift = fit_params[0][0]
    ax.plot(xaxis,np.exp(fit_shift+fit_slope*xaxis),c="red",lw=3,label=r"$e^{{{0}+{1}N_{{\mathrm{{steps}}}}}}$".format(round(fit_shift,3),round(fit_slope,3)))

    load_filename = FILENAME+'_plot_cor_mid_stepstotal='+str(max(list(cur_dw_int_dict.keys())))
    plt.tight_layout()
    if IS_PBC:
        load_filename += "_pbc"
    # Change tick sizes
    ax.tick_params(axis = 'both', which = 'major', labelsize = FONTSIZE_TICKS)
    
    ax.legend(loc='upper right',fontsize=FONTSIZE_LEGEND+4)
    ax.figure.savefig(load_filename+'.pdf', bbox_inches='tight')    

In [ ]:
def get_dict_sim(cur_nsteps,sys_size,choose_steps=None,noise_model=None):
    if choose_steps is None:
        choose_steps = cur_nsteps-7
    QUBITS_NUM = sys_size
    TODAY = str(date.today())
    FILENAME = r'C:\Users\USER\Desktop\Backup\code_backup_dms\KZ_sim'+'\\counts_z2_1system_dist_'+RUN_TYPE+'_'+str(QUBITS_NUM)
    print(FILENAME)
    try:
        fname_toload = FILENAME+'_'+str(choose_steps)+'_stepstotal='+str(cur_nsteps)+".npy"
        cur_resdws_dict = np.load(fname_toload,allow_pickle=True).item()
    except Exception:
        fname_toload = FILENAME+'_'+str(choose_steps)+'_stepstotal='+str(cur_nsteps)+'_noise_model='+str(noise_model)+".npy"
        cur_resdws_dict = np.load(fname_toload,allow_pickle=True).item()       
    return cur_resdws_dict

In [ ]:
# TODO: Move to imports
from dataclasses import dataclass

# TODO: ADD HERE THE CODE FOR CALCULATING EXPLICITELY THE PROBABILITY OF <s_i|\psi> using ED and qiskit. Plot graph.
# Run the adiabatic sweeping
# We verfied that this definition matches the L=4 verfication_results

@dataclass
class NoiseParameters:
    each_step_noise: float
    disorder: float
    
def get_beta_arr(N_steps,sys_size,skip_steps=20,noise_model=None):
    global QUBITS_NUM
    global FILENAME
    global TODAY
    QUBITS_NUM = sys_size
    TODAY = str(date.today())
    FILENAME = 'counts_z2_1system_dist_'+RUN_TYPE+'_'+str(sys_size)
    r0=1 # For some reason we use 0.5 factor in the free fermion definitions
    betas_thermal = dict()
    betas_dist = dict()
    beta_from_dw_arr = dict()
    dw_int_dict = dict()
    dw_int_minus_avg_dict = dict()

    # Initializing disorder noise to be the same for all steps
    if noise_model:
        disorder_alpha = np.random.normal(0, noise_model.disorder, sys_size)
        disorder_beta = np.random.normal(0, noise_model.disorder, sys_size)

    for steps in range(3,N_steps+1,skip_steps):
        #if steps<N_steps:
        #    continue
        # Create a Quantum Register with QUBITS_SYSTEM_NUM>?QUBITS_NUM qubits.
        regq = QuantumRegister(sys_size)
        c = ClassicalRegister(sys_size)
        # Create a Quantum Circuit acting on the q register
        outcirc = QuantumCircuit(regq, c)

        # Prepare the ground state of the initial state, which is the X basis
        prepare_plus_state(outcirc,regq)

        # New N_steps definition - Copy pasted rom the free-fermion method
        steps_list = np.linspace(0,np.pi/2,steps+2)[1:-1]

        # Evolution with F(alpha,beta) adiabatically for N_{steps}
        for cur_step in steps_list:
            theta = cur_step
            # The half here seems to be artifact of some notation
            # TODO: ADD NOISE HERE
            alpha_cur = r0 * np.cos(theta) # H_{PM} coefficient
            beta_cur = r0*np.sin(theta) # H_{FM} coefficient
            if noise_model:
                alpha_arr = np.zeros(sys_size)
                if not IS_PBC:
                    beta_arr = np.zeros(sys_size-1)
                else:
                    beta_arr = np.zeros(sys_size)
                # Noise
                alpha_arr += np.random.normal(0, noise_model.each_step_noise, sys_size)
                beta_arr += np.random.normal(0, noise_model.each_step_noise, sys_size)
                # Disorder
                alpha_arr += disorder_alpha
                beta_arr += disorder_beta
                apply_floquet_arr(outcirc,regq,alpha_arr+alpha_cur,beta_arr+beta_cur)
            # TODO: Make a version for apply_floquet with array instead of one variable
            else:
                apply_floquet(outcirc,regq,alpha_cur,beta_cur)
        outcirc.save_statevector()
        #measure_partial_system(outcirc,regq, c)
        fname_save = FILENAME+'_'+str(steps)+'_stepstotal='+str(N_steps)+'_noise_model='+str(noise_model)
        res_dws,dw_exp,dw_int,dw_int_minus_avg = run_circ_measure_prob_dist(outcirc,SIM_NUM,BATCH_SIZE, RUN_TYPE, fname_save)
        # Save the array first
        np.save(r"C:\Users\USER\Desktop\Backup\code_backup_dms\KZ_sim"+"\\"+fname_save+".npy", res_dws)
        cur_thermal_beta_pair = get_beta([res_dws],fit_op = GetBetaOption.thermal_fit)
        cur_dist_beta_pair = get_beta([res_dws],fit_op = GetBetaOption.lin_dist_fit)
        betas_thermal[steps] = cur_thermal_beta_pair
        betas_dist[steps] = cur_dist_beta_pair
        dw_int_dict[steps] = dw_int
        dw_int_minus_avg_dict[steps] = dw_int_minus_avg
        print(cur_thermal_beta_pair)
        print(cur_dist_beta_pair)

        beta_from_dw_arr[steps] = -0.5*np.log(dw_exp/(1-dw_exp))
        
        
        #plot_ener_dist_space([res_dws])
    return betas_thermal,betas_dist,beta_from_dw_arr,dw_int_dict,dw_int_minus_avg_dict
    

In [ ]:
noise_pa = NoiseParameters(each_step_noise=0.0,disorder=0)
glob_N_steps = 12
sys_size_arr = [14]#[20]
dw_res_dist_arr = [(x,get_beta_arr(glob_N_steps,x,1,noise_pa)) for x in sys_size_arr]

In [ ]:
# Plot \beta(N_steps) with varying N_steps and constant system size
glob_N_steps = 500
sys_size_arr = [14]#[20]
dw_res_dist_arr = [(x,get_beta_arr(glob_N_steps,x,20)) for x in sys_size_arr]

In [ ]:
#steps_choice = range(3,50,4)
steps_choice = None
plot_dw_int(dw_res_dist_arr[0][1][3],steps_choice,plot_type=DWType.dw_int)
plot_dw_int(dw_res_dist_arr[0][1][4],steps_choice,plot_type=DWType.dw_int_minus_avg)

In [ ]:
plot_cor_mid(dw_res_dist_arr[0][1][4],steps_choice,plot_type=DWType.dw_int_minus_avg)
# Clearly exponential fit should work here

In [ ]:
N_steps=3
print(sys_size_arr[0])
loaded_sim_dict = get_dict_sim(glob_N_steps,sys_size_arr[0],N_steps,noise_pa)
print(len(list(loaded_sim_dict.keys())[0]))

In [ ]:
plot_ener_dist_space([loaded_sim_dict],GetBetaOption.thermal_fit,noise_pa)

In [ ]:
# This is the plot for \beta(N_steps)
fig, ax = plt.subplots(figsize=(10, 10))

syssize_arr = [x[0] for x in dw_res_dist_arr]
for cur_sys_size,cur_betas in dw_res_dist_arr:
    if cur_sys_size<15:
        continue
    cur_thermal_betas = cur_betas[0]
    cur_dist_betas = cur_betas[1]
    cur_dw_betas = cur_betas[2]
    xaxis = sorted(cur_thermal_betas.keys())[1:]
    yaxis_thermal = [cur_thermal_betas[x][0] for x in xaxis]
    yerr_thermal_arr = [cur_thermal_betas[x][1] for x in xaxis]
    yaxis_dist = [cur_dist_betas[x][0] for x in xaxis]
    yerr_dist_arr = [cur_dist_betas[x][1] for x in xaxis]
    yaxis_dw = [cur_dw_betas[x] for x in xaxis]
    #print(dw)
    #yaxis_dw = [np.log(np.exp(-2*cur_dw_betas[x])/(1-np.exp(-2*cur_dw_betas[x])))/(-2) for x in xaxis]
    

    fit_params = scipy.optimize.curve_fit(lambda x,a,b: a+b*x, np.log(np.array(xaxis[:])[:]), np.log(np.array(yaxis_dist[:])[:]))
    a = fit_params[0][0]
    b = fit_params[0][1]
    #print('yaxis:'+str(np.log(np.array(yaxis[:])[:])))
    print('{0}+{1}*x'.format(str(round(a,5)), str(round(b,5))))
    
    ax.errorbar(xaxis,yaxis_thermal,ls='--',marker='o',markersize=4,capsize=2,yerr=yerr_thermal_arr,label=r"$L={0}$ from thermal fit".format(cur_sys_size))
    ax.errorbar(xaxis,yaxis_dist,ls='--',marker='o',markersize=4,capsize=2,yerr=yerr_dist_arr,label=r"$L={0}$ from dist. fit".format(cur_sys_size))
    ax.plot(xaxis,yaxis_dw,ls='-',marker='o',markersize=4,label=r"$L={0}$ from DW fit".format(cur_sys_size))
    
    xaxis_fit = np.log(np.array(xaxis[:])[:])
    #ax.plot(xaxis,np.exp(a+b*xaxis_fit),ls='-',markersize=4)
    if cur_sys_size==20:
        ax.plot(xaxis,0.01+0.5*np.log(xaxis),ls='-',lw=3)
# Set log scale
#ax.set_yscale('log')
ax.set_xscale('log')

ax.set_ylabel(r'$\beta$',fontsize=FONTSIZE_AXIS)
ax.set_xlabel(r'$N_{\rm steps}$',fontsize=FONTSIZE_AXIS)
# TODO: Unique names
name = 'qiskit_beta_as_function_of_nsteps_L={0}'.format(str(syssize_arr))
if IS_PBC:
    name += "_pbc"

# Change tick sizes
ax.tick_params(axis = 'both', which = 'major', labelsize = FONTSIZE_TICKS)
ax.tick_params(axis = 'both', which = 'minor', labelsize = FONTSIZE_TICKS)

ax.legend(loc='upper left',fontsize=FONTSIZE_LEGEND)
ax.figure.savefig(name+'.png')
ax.figure.savefig(name+'.pdf')

The graph here shows the $\beta$ is not the same as the distribution is not exactly termal. Actually references show that the distribution of kinks is approximaetly normal. See papers by DWave and Adolfo.

$\sum_i P(Z_iZ_{i+1}=1,Z_{j\neq i}Z_{j+1}=-1) = \sum_i $

## Quantum simulation of the domain walls defects

In [ ]:
sys_size = 16

In [ ]:
# Run the adiabatic sweeping
# We verfied that this definition matches the L=4 verfication_results
N_steps = 800
#dt = 0.1
r0=1 # For some reason we use 0.5 factor in the free fermion definitions

means = []
devs = []

for steps in range(1,N_steps+1,20):
    # Create a Quantum Register with QUBITS_SYSTEM_NUM>?QUBITS_NUM qubits.
    regq = QuantumRegister(sys_size)
    c = ClassicalRegister(sys_size)
    # Create a Quantum Circuit acting on the q register
    outcirc = QuantumCircuit(regq, c)

    # Prepare the ground state of the initial state, which is the X basis
    prepare_plus_state(outcirc,regq)
    
    #curT = 0.5*dt*steps
    # New N_steps definition - Copy pasted rom the free-fermion method
    steps_list = np.linspace(0,np.pi/2,steps+2)[1:-1]

    # Evolution with F(alpha,beta) adiabatically for N_{steps}
    for cur_step in steps_list:
        #curg = -1*(2-(cur_step*dt)/curT)
        #curJ = -1*(cur_step*dt)/curT
        #s = float(cur_step_num)/float(N_steps)
        #alpha_cur = curg*dt # H_{PM} coefficient
        #beta_cur =  curJ*dt # H_{FM} coefficient
        theta = cur_step
        # NO HALF HERE AS WE CORRECTED THE NOTATION
        alpha_cur = r0 * np.cos(theta) # H_{PM} coefficient
        beta_cur = r0*np.sin(theta) # H_{FM} coefficient
        apply_floquet(outcirc,regq,alpha_cur,beta_cur)

    # Measure and get from it the domain wall excitations
    #measure_partial_system(outcirc,regq, c)
    #outcirc.draw()
    #mean, stddev = run_circ_measure_dw(outcirc,SIM_NUM,BATCH_SIZE, RUN_TYPE, FILENAME+'_'+str(steps)+'_stepstotal='+str(N_steps))
    #means.append(mean)
    #devs.append(stddev)

    outcirc.save_statevector()
    res = run_circ_measure_dw_probs(outcirc)
    means.append(res)
    devs.append(0)

print(means)
print(devs)
#apply_floquet(withf_circ,regq,alpha_cur,beta_cur)
#withf_circ.draw()
#run_circ(withf_circ,SIM_NUM,BATCH_SIZE, RUN_TYPE, FILENAME+'_afterf')

### Plot DW density graph

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
xaxis = list(range(2,N_steps+1,20))
ax.set_xticks(xaxis) # NO MINOR BOOL FOR NEWER MATPLOTLIB
ax.set_ylabel(r'Excitations density $\frac{N_{\rm ex}}{L}$',fontsize=FONTSIZE_AXIS)
ax.set_xlabel(r'$N_{\rm steps}$',fontsize=FONTSIZE_AXIS)

# Set log scale
ax.set_yscale('log')
ax.set_xscale('log')  

l_yaxis = means
l_yerr_arr = devs
#l_yerr_arr = np.zeros(len(l_yerr_arr))
ax.errorbar(xaxis,l_yaxis,ls='--',label=r'$\sigma=0$',marker='o',markersize=4,capsize=2,yerr=l_yerr_arr)

print(xaxis[-1])
print(l_yaxis[-1])

# TODO: Unique names
name = 'kz_scaling_'+str(QUBITS_NUM)+'system_qiskit_debug_to_delete'

# Change tick sizes
ax.tick_params(axis = 'both', which = 'major', labelsize = FONTSIZE_TICKS)
ax.tick_params(axis = 'both', which = 'minor', labelsize = FONTSIZE_TICKS)

ax.legend(loc='lower left',fontsize=FONTSIZE_LEGEND)
#ax.figure.savefig(name+'.png')
#ax.figure.savefig(name+'.svg')

### Plot the matching $\beta$ graph

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
xaxis = list(range(2,N_steps+1,20))
#ax.set_xticks(xaxis) # NO MINOR BOOL FOR NEWER MATPLOTLIB
ax.set_ylabel(r'$\beta$',fontsize=FONTSIZE_AXIS)
ax.set_xlabel(r'$N_{\rm steps}$',fontsize=FONTSIZE_AXIS)

# Set log scale
#ax.set_yscale('log')
#ax.set_xscale('log')  

# Assuming \beta is large enough
l_yaxis = np.log(means)/(-2)
l_yerr_arr = devs
#l_yerr_arr = np.zeros(len(l_yerr_arr))
ax.errorbar(xaxis,l_yaxis,ls='--',label=r'$\sigma=0$',marker='o',markersize=4,capsize=2,yerr=l_yerr_arr)

print(xaxis[-1])
print(l_yaxis[-1])

# TODO: Unique names
name = 'kz_scaling_'+str(QUBITS_NUM)+'system_qiskit_debug_to_delete'

# Change tick sizes
ax.tick_params(axis = 'both', which = 'major', labelsize = FONTSIZE_TICKS)
ax.tick_params(axis = 'both', which = 'minor', labelsize = FONTSIZE_TICKS)

ax.legend(loc='lower left',fontsize=FONTSIZE_LEGEND)
#ax.figure.savefig(name+'.png')
#ax.figure.savefig(name+'.svg')